# Lesson 30: Building LLM Judge Metrics for AI Evals

In this lesson, we'll build LLM judge metrics to evaluate the quality of articles generated by Brown, the writing workflow.

**Learning Objectives:**

- Understand how to build LLM judges that evaluate articles at the section level using binary metrics
- Learn the `FollowsGTMetricLLMJudge` metric that compares generated articles to ground truth
- Learn the `UserIntentMetricLLMJudge` metric that checks guideline adherence and research anchoring
- Use the evaluation dataset created in Lesson 28 to run experiments
- Execute end-to-end AI evals using Opik and the Brown agent


> [!NOTE]
> 💡 Remember that you can also run `brown` as a standalone Python package by going to `lessons/writing_workflow/` and following the instructions from there. We have a script at `lessons/writing_workflow/scripts/brown_run_eval.py` that you can use to run evaluations as well.


## 1. Setup

First, we define some standard Magic Python commands to autoreload Python packages whenever they change:


In [1]:
%load_ext autoreload
%autoreload 2

### Set Up Python Environment

To set up your Python virtual environment using `uv` and load it into the Notebook, follow the step-by-step instructions from the `Course Admin` lesson from the beginning of the course.

**TL/DR:** Be sure the correct kernel pointing to your `uv` virtual environment is selected.


### Configure Gemini and Opik API's

To configure the Gemini and Opik API, follow the step-by-step instructions in the `Course Admin` lesson.

Here is a quick checklist of what you need to run this notebook:

1.  Get your key from [Google AI Studio](https://aistudio.google.com/app/api-keys).
2.  Get your key from [Opik](https://www.comet.com/site/products/opik/).
3.  From the root of your project, run: `cp .env.example .env` 
4.  Within the `.env` file, fill in the `GOOGLE_API_KEY` and `OPIK_API_KEY` variables.

Now, the code below will load the keys from the `.env` file:


In [2]:
from utils import env

env.load(required_env_vars=["GOOGLE_API_KEY", "OPIK_API_KEY"])

Environment variables loaded from `/Users/pauliusztin/Documents/01_projects/TAI/agentic-ai-engineering-course/.env`
Environment variables loaded successfully.


### Import Key Packages


In [3]:
import nest_asyncio
from utils import pretty_print

nest_asyncio.apply()  # Allow nested async usage in notebooks

### Download Required Files

First, let's download Brown's configs folder:


In [4]:
%%capture

!rm -rf configs
!curl -L -o configs.zip https://raw.githubusercontent.com/iusztinpaul/agentic-ai-engineering-course-data/main/data/configs.zip
!unzip configs.zip
!rm -rf configs.zip

Now, let's download Brown's inputs folder containing profiles, examples, and test data:

In [5]:
%%capture

!rm -rf inputs
!curl -L -o inputs.zip https://raw.githubusercontent.com/iusztinpaul/agentic-ai-engineering-course-data/main/data/inputs.zip
!unzip inputs.zip
!rm -rf inputs.zip

Ultimately, we will download Brown's outputs folder that contains pre-generated articles used as a cache to speed things up and keep costs low:

In [6]:
%%capture

!rm -rf outputs
!curl -L -o outputs.zip https://raw.githubusercontent.com/iusztinpaul/agentic-ai-engineering-course-data/main/data/outputs.zip
!unzip outputs.zip
!rm -rf outputs.zip

Let's verify what we downloaded:


In [7]:
%ls


article.md                  notebook.ipynb
article_guideline.md        notebook_guideline.md
article_guideline_notes.md  outputs/
configs/                    research.md
inputs/


Also, create constants for all the directories of interest:

In [8]:
from pathlib import Path

CONFIGS_DIR = Path("configs")
INPUTS_DIR = Path("inputs")
OUTPUTS_DIR = Path("outputs")

print(f"Configs directory exists: {CONFIGS_DIR.exists()}")
print(f"Inputs directory exists: {INPUTS_DIR.exists()}")
print(f"Outputs directory exists: {OUTPUTS_DIR.exists()}")


Configs directory exists: True
Inputs directory exists: True
Outputs directory exists: True


In [9]:
SAMPLE_DIR = Path("inputs/tests/01_sample_small")

print(f"Sample directory exists: {SAMPLE_DIR.exists()}")

Sample directory exists: True


## 2. Understanding Our LLM Judge Metrics

Before diving into the code, let's understand what LLM judges we will build and how they work with our evaluation dataset.

### The Evaluation Dataset Structure

Remember that in Lesson 28, we created an evaluation dataset where each sample contains:
- **Article Guideline**: The user intent describing how the article should look
- **Research**: The source material the article should be based on  
- **Expected Article (Ground Truth)**: The reference article to compare against

The key insight is that **the generated article is computed on the fly**. Every time we make changes to our AI app (Brown), we regenerate the articles and recompute the metrics. This is why everything else is static—we only vary the generated outputs and system parameters during optimization.

<img src="https://raw.githubusercontent.com/iusztinpaul/agentic-ai-engineering-course-data/main/images/30_ai_evals_offline_metrics_dataset_samples.png" alt="30_ai_evals_offline_metrics_dataset_samples" height=600/>


### The Two LLM Judge Metrics

We will build two complementary LLM judge metrics:

1. **FollowsGT (Follows Ground Truth)**: Compares the **generated article** with the **expected article** across three dimensions:
   - **Content**: Does the generated section cover the same topics and ideas?
   - **Flow**: Does it follow the same order of ideas and transitions?
   - **Structure**: Does it use the same formatting patterns (headers, lists, code blocks)?

2. **UserIntent**: Compares the **generated article** with the **input** (article guideline and research) across two dimensions:
   - **Guideline Adherence**: Does the generated article follow the specific requirements from the article guideline?
   - **Research Anchoring**: Is the content based solely on the provided research, or does it hallucinate?

### Section-Level Binary Metrics

Our LLM judges use **binary metrics scoped at the section level** instead of the whole article level. For each article, we compute all the metrics from above (content, flow, structure, guideline adherencem research anchoring) for each section individually.

**Why this approach?**
- **More signal**: Binary metrics at the section level give us granular insights into what went wrong
- **Easier calibration**: It's easier to align binary (0/1) judgments with human expectations
- **Independence**: For all our metrics, sections are independent units

<img src="https://raw.githubusercontent.com/iusztinpaul/agentic-ai-engineering-course-data/main/images/30_ai_evals_offline_metrics_section_level.png" alt="30_ai_evals_offline_metrics_section_level" height=600/>


## 3. Dataset Splits for LLM Judge Development

We split our dataset strategically to develop and validate our LLM judges:

- **Training Split (Few-Shot Examples)**: Used as few-shot examples to "train" the LLM judges. These manually labeled examples teach the judge how to score correctly.
- **Validation Split**: Used to align the LLM judges with human expectations. We evaluate the judges themselves on this split.
- **Testing Split**: Used to compute the final metrics on unseen data.

<img src="https://raw.githubusercontent.com/iusztinpaul/agentic-ai-engineering-course-data/main/images/30_ai_evals_offline_metrics_dataset_splits.png" alt="30_ai_evals_offline_metrics_dataset_splits" height=600/>

Remember that in the previous lesson, where we constructed the AI evals dataset, Lessons 4 and 7 are marked as `is_few_shot_example=True`, meaning we will use these two lessons as few-shot examples to "train" our LLM judges.


## 4. The FollowsGT Metric: Comparing Against Ground Truth

Let's dive into the `FollowsGTMetricLLMJudge`, our first LLM judge metric. We'll examine the base abstractions used across all the metrics and then the specific implementation.

### 4.1 The BrownBaseMetric Abstraction

All our LLM judge metrics inherit from `BrownBaseMetric`, which provides common functionality for model initialization, structured output parsing, and scoring interfaces.

Source: `brown.evals.metrics.base`

```python
class BrownBaseMetric(base_metric.BaseMetric, Generic[FewShotExamplesT, StructuredOutputTypeT], abc.ABC):
    """Abstract base class for Brown evaluation metrics that use LLMs for structured scoring."""

    def __init__(
        self,
        model: SupportedModels,
        name: str,
        structured_output_type: type[StructuredOutputTypeT],
        few_shot_examples: FewShotExamplesT,
        model_config: ModelConfig,
        track: bool = True,
        project_name: str | None = None,
    ) -> None:
        super().__init__(
            name=name,
            track=track,
            project_name=project_name,
        )

        self.model = model
        self.structured_output_type = structured_output_type
        self.model_config = model_config
        self.few_shot_examples = few_shot_examples

        if self.model == SupportedModels.FAKE_MODEL:
            assert self.model_config and self.model_config.mocked_response is not None

    def init_model(self) -> Runnable:
        """Initialize the language model with structured output capabilities."""
        model_instance = get_model(self.model, self.model_config)
        model_instance = model_instance.with_structured_output(self.structured_output_type)
        return model_instance

    def score(self, *args: Any, **kwargs: Any) -> score_result.ScoreResult | list[score_result.ScoreResult]:
        """Calculate evaluation scores synchronously by wrapping the async implementation."""
        return a.asyncio_run(self.ascore(*args, **kwargs))

    @abc.abstractmethod
    async def ascore(self, *args: Any, **kwargs: Any) -> score_result.ScoreResult | list[score_result.ScoreResult]:
        """Abstract async method for implementing metric-specific evaluation logic."""
        pass
```

**Key Features:**
- **Generic Type Parameters**: `FewShotExamplesT` for few-shot examples and `StructuredOutputTypeT` for the LLM response format
- **Model Initialization**: `init_model()` creates a fresh model instance with structured output for each evaluation
- **Sync/Async Support**: `score()` wraps the async `ascore()` method for synchronous usage


### 4.2 The FollowsGTMetricLLMJudge Implementation

Now let's look at the concrete implementation:

Source: `brown.evals.metrics.follows_gt.metric`

```python
class FollowsGTMetricLLMJudge(BrownBaseMetric):
    """A metric that evaluates the quality of article content across multiple sections and dimensions."""

    def __init__(
        self,
        model: SupportedModels = SupportedModels.GOOGLE_GEMINI_25_FLASH,
        name: str = "follows_gt",
        model_config: ModelConfig | None = None,
        track: bool = True,
        project_name: str | None = None,
    ) -> None:
        model_config = model_config or ModelConfig(
            temperature=0.0, thinking_budget=1024 * 4, include_thoughts=False, max_retries=3
        )
        super().__init__(
            model=model,
            name=name,
            structured_output_type=FollowsGTArticleScores,
            few_shot_examples=prompts.DEFAULT_FEW_SHOT_EXAMPLES,
            model_config=model_config,
            track=track,
            project_name=project_name,
        )

    async def ascore(
        self,
        output: str,
        expected_output: str,
        **ignored_kwargs: Any,
    ) -> score_result.ScoreResult | list[score_result.ScoreResult]:
        # Initialize the model client at the function level to avoid coroutine reuse issues
        model_client = self.init_model()

        llm_query = prompts.get_eval_prompt(
            output=output,
            expected_output=expected_output,
            few_shot_examples=self.few_shot_examples,
        )
        article_response = cast(
            FollowsGTArticleScores,
            await model_client.ainvoke([{"role": "user", "content": llm_query}]),
        )

        if not article_response:
            raise ValueError("Model failed to return a structured response.")

        return article_response.to_score_result(self.name)
```

**Key Points:**
- Uses `FollowsGTArticleScores` as the structured output type
- Takes `output` (generated article) and `expected_output` (ground truth) as inputs
- Returns a list of `ScoreResult` objects, one per dimension (content, flow, structure)

Still, the real magic happens in how we define our `FollowsGTArticleScores` scoring entities and within the prompt. Let's start with the structured output entities. 


### 4.3 The Score Types Hierarchy

The structured output uses a hierarchy of Pydantic models:

Source: `brown.evals.metrics.follows_gt.types`

1. **FollowsGTArticleScores** - Top-level container for all section scores:

```python
class FollowsGTArticleScores(pydantic.BaseModel):
    """Article-level scores for the FollowsGT evaluation metric."""

    sections: list[FollowsGTSectionScores]

    def to_score_result(self, prefix: str) -> list[score_result.ScoreResult]:
        """Convert the evaluation results to ScoreResult objects with dimension-wise scoring."""
        return aggregate_section_scores_to_results(self.sections, prefix)
```

2. **FollowsGTSectionScores** - Scores for a single section:

```python
class FollowsGTSectionScores(pydantic.BaseModel):
    """Section evaluation with scores across all FollowsGT dimensions."""

    title: str = pydantic.Field(description="The title of the section being evaluated.")
    scores: FollowsGTCriterionScores = pydantic.Field(description="The scores of the section.")
```

3. **FollowsGTCriterionScores** - The three dimension scores:

```python
class FollowsGTCriterionScores(CriteriaScores):
    """Represents scores for all three evaluation dimensions of a section."""

    content: CriterionScore
    flow: CriterionScore
    structure: CriterionScore
```

The only custom abstractions appear within the `CriteriaScores` Pydantic model. Let's see how that works.

### 4.4 Base Score Classes

Source: `brown.evals.metrics.base`

1. **CriterionScore** - We first define a single binary score with it's reasoning trace:

```python
class CriterionScore(pydantic.BaseModel):
    """Base model for a single score representing a specific evaluation dimension."""

    score: Annotated[int, Ge(0), Le(1)] = pydantic.Field(description="Binary score of the section.")
    reason: str = pydantic.Field(description="The reason for the given score.")
```

2. **CriteriaScores** - Then we define a base class that aggregates multiple scores within a single unit and knows how to translate it to context for LLMs. We used this to define the `FollowsGTCriterionScores` from above, which are used for each section of one article:

```python
class CriteriaScores(pydantic.BaseModel):
    """Abstract base class for scores across multiple evaluation dimensions."""

    def to_context(self) -> str:
        """Convert the scores to a formatted XML string for use as context in prompts."""
        scores_fields = self.__class__.model_fields
        scores_xml = ""
        for field_name in scores_fields.keys():
            field_score = getattr(self, field_name)
            scores_xml += f"""    <{field_name}>
        <score>{field_score.score}</score>
        <reason>{field_score.reason}</reason>
    </{field_name}>
"""
        return scores_xml
```


### 4.5 The Aggregation Function

The `aggregate_section_scores_to_results` function is the key to converting section-level scores to final metrics:

Source: `brown.evals.metrics.base`

```python
def aggregate_section_scores_to_results(
    section_scores: list,
    prefix: str,
) -> list[score_result.ScoreResult]:
    """Convert section-level evaluation results to aggregated ScoreResult objects."""
    if not section_scores:
        return []

    # Automatically infer dimensions from the first section's scores class
    scores_class = type(section_scores[0].scores)
    scores_fields = scores_class.model_fields
    aggregated_scores: dict[str, dict[str, list[int] | str]] = {
        field_name: {"scores": [], "reason": ""}
        for field_name in scores_fields.keys()
    }

    for section in section_scores:
        for dimension in aggregated_scores.keys():
            dimension_score = getattr(section.scores, dimension)
            aggregated_scores[dimension]["scores"].append(dimension_score.score)
            aggregated_scores[dimension]["reason"] += f"{section.title}:\n"
            aggregated_scores[dimension]["reason"] += f"**{dimension_score.score}:** {dimension_score.reason}\n\n"

    results: list[score_result.ScoreResult] = []
    for dimension, scores_data in aggregated_scores.items():
        scores_list = scores_data["scores"]
        aggregated_score = CriterionAggregatedScore(
            name=f"{prefix}_{dimension}",
            score=sum(scores_list) / len(scores_list),
            reason=str(scores_data["reason"]),
        )
        results.append(aggregated_score.to_score_result())

    return results
```

**How it works:**
1. **Infer dimensions**: Automatically detects score dimensions (content, flow, structure) from the Pydantic model
2. **Collect scores**: Gathers binary scores and reasons from each section
3. **Compute averages**: Calculates the mean score per dimension across all sections. As each is a binary score (0 or 1), the average will result in a number within the `[0, 1]` interval.
4. **Build reasons**: Concatenates section-level reasons for debugging
5. **Return ScoreResults**: Produces one `ScoreResult` per dimension (e.g., `follows_gt_content: 0.75`, `follows_gt_flow: 0.6`)

We used this function within the `FollowsGTArticleScores` class to transform the list of scores for each section into a single aggregate score for the entire article.


### 4.6 Few-Shot Examples Classes

The few-shot examples teach the LLM judge how to score correctly. 

Remember how we split the dataset between training, validation and testing? With these classes we model the training split passed to the LLM judge within the system prompt.

Source: `brown.evals.metrics.follows_gt.types`

```python
class FollowsGTMetricFewShotExample(BaseExample):
    """Represents a single example for the follows_gt evaluation."""

    output: str
    expected_output: str
    scores: FollowsGTArticleScores

    @classmethod
    def from_markdown(
        cls, output_file: Path, expected_output_file: Path, scores: FollowsGTArticleScores
    ) -> "FollowsGTMetricFewShotExample":
        output = output_file.read_text()
        expected_output = expected_output_file.read_text()
        return cls(output=output, expected_output=expected_output, scores=scores)

    def to_context(self) -> str:
        return f"""
<output>
{self.output}
</output>
<expected_output>
{self.expected_output}
</expected_output>
{self.scores.to_context()}
"""


class FollowsGTMetricFewShotExamples(BaseFewShotExamples[FollowsGTMetricFewShotExample]):
    """Collection of few-shot examples for the FollowsGT evaluation metric."""
    pass
```

And the base classes that define the common interface and functionality of the few-shot examples:

Source: `brown.evals.metrics.base`

```python
class BaseExample(pydantic.BaseModel, Generic[ExampleT]):
    """Base class for examples used in evaluation metrics."""

    @abc.abstractmethod
    def to_context(self) -> str:
        """Convert the example to a formatted string for use as context in prompts."""
        pass


class BaseFewShotExamples(pydantic.BaseModel, Generic[ExampleT]):
    """Base class for few-shot examples collections."""

    examples: list[ExampleT]

    def to_context(self) -> str:
        examples = "\n\n".join(
            [f"<example_{i + 1}>\n\t{example.to_context()}\n</example_{i + 1}>\n" 
             for i, example in enumerate(self.examples)]
        )
        return examples
```

### 4.7 The System Prompt

The system prompt is carefully engineered to guide the LLM judge in scoring each section of the article using our metrics: content, flow, and structure. 

Here is the prompt divided by key sections:

Source: `brown.evals.metrics.follows_gt.prompts`

**1. Introduction:** - Sets the role and general task:
```python
SYSTEM_PROMPT = """
You are an expert in Natural Language Processing (NLP) evaluation metrics, specifically trained to 
assess answer quality in responses provided by large language models (LLMs). 

Your task is to evaluate the quality of a generated article by another LLM relative to 
an expected article output across multiple criteria: content, flow, structure, and mechanics.
...
"""
```

**2. Scoring Instructions** - Section-level binary scoring across the content, flow and structure dimensions. Note how detailed we defined each step of the LLM judge on how to use the expected and generated outputs, plus how to reason about the output business metrics:
```python
"""
...
## INSTRUCTIONS 

1. You must analyze the given expected article (<expected_output>) and generated article (<generated_output>) 
to determine the most relevant evaluation.
2. Since the generated output is an answer from another LLM, you must use the expected output as the reference 
standard to compare and evaluate the quality of the generated output.
3. Both the generated and expected outputs are in Markdown format.
4. Instead of comparing the outputs as a whole, you will divide the outputs into sections and compare each section 
individually. 
5. You will always use the expected output as the reference point to extract the sections of interest during the
evaluation. If there is no perfect match between the expected and generated section names, first try to infer
the corresponding section based on the similarity of section names and their respective content. If you conclude that 
the expected output contains a section that the generated output lacks, you will assign a score of 0 to the missing 
section in the generated output.
6. Sections are divided by H2 headers, marked as "##" in Markdown. You will use these headers as 
separators. Anything between two H2 headers constitutes a section. The only valid exception to this rule is the first 
section, the introduction, which sometimes appears between the title and the first H2 header. You will never include 
the title or subtitle as part of the first section.
7. When comparing each individual section of the expected output to the generated output, you will assign a binary 
score for multiple criteria: 0 or 1, where 0 indicates a non-match and 1 indicates a perfect match. Each criterion
is completely independent of the others, meaning that a score of 0 in one criterion does not affect the score of 
another criterion.
8. You must compute binary scores for each section based on the following criteria:
  1. **Content:** Evaluate whether the generated section covers the same content as the expected section:
    - Focus only on evaluating that the substance of the content is the same between the expected and generated section. By content, we 
    mean core subjects, topics, research, ideas, key points or arguments. For example, if both sections discuss the
    fundamentals of RAG, it's valid. But, if the expected section discusses advanced RAG topics, while the generated section
    discusses basic RAG topics, it's invalid.
    - In this criterion, we are not interested in the order, structure, layout, or any other aspect related to the flow of
    ideas, structure or mechanics. For example, if there are missing or additional ideas discussed it's still valid, as
    long as the substance of the content between the expected and generated section is the same.
  2. **Flow:** Evaluate whether the generated section follows the same order of ideas as the expected section, such as 
  the flow of:
    - Main ideas covered starting with the beginning, until the end of the section. With special emphasis on the beginning and end of the 
    section as they reflect the transition between the previous and next sections
    - Internal transitions between the main points within the section. We expect a smooth flow of ideas, 
    without any abrupt jumps or breaks.
    - Placement of notes, images, tables, code blocks, or any other media elements within the generated section, 
    relative to the expected section. 
    - We don't expect a perfect one on one match between the paragraphs and sentences between the expected and generated section.
    However, we expect the same ideas and concepts to be discussed in the same way, order, and storyline.
    - Assign a score of 0 if anything is missing from the generated section relative to the expected section, such as missing
    topics, ideas, or media.
    - Assign a score of 0 if anything is additionally added to the generated section relative to the expected section, such 
    as additional topics, ideas or media elements.
    - Accepted differences between the expected and generated section:
        - Mismatching media numbering is accepted. For example, if in the expected section we have a figure with the number 3 and
        in the generated section we have a figure with the number 4, it's valid. It will be invalid, only if the figure would be
        missing altogether.
        - Mismatching or missing emojis. For example, if the excepted section has a 💡 emoji, while the generated section has 
        a 🔑 emoji, it's valid. Also, if the emoji is missing altogether from the generated section, it's valid.
        - Mismatching source reference numbers. For example, if the expected section referes a source with the number 3, 
        while the generated section referes a source with the number 7, it's valid. It will be invalid, only if the generated
        section misses the source altogether. 
        - Different placement of the source in the generated section. For example, if the expected section has the source
        at the end of a sentence within the paragraph, while the generated section has it at the end of the paragraph, it's valid. 
        It will be invalid, only if the generated section would be missing altogether.
        - Mismatching number of source references. For example, if the expected section has 3 source references, while the 
        generated section has 2 source references, it's valid. It will be invalid, only if the generated section would have 
        misses the references altogether. For example if the expected section has 3 source references, while the generated 
        section has 0 source references, it's invalid.
        - Having reference numbers in the generated section, while having none in the expected section. For example, if the 
        expected section has 0 reference numbers, while the generated section has 3 reference number, it's valid. It will be 
        invalid, only the other way around, where the expected section has 3 reference numbers, while the generated section has 0.
  3. **Structure:** Evaluate whether the generated section follows the same structure as the expected section. By 
  structure, we mean:
    - H3/H4/H5/H6 sub-heading structure and formatting
    - Mismatches in headers formatting and presence. For example, if the expected section doesn't has a header,
    while the generated section has one, it's invalid. It's valid only if there is a one on one match between the headers
    formatting and presence.
    - Use of bulleted lists, numbered lists, callouts, notes, or other layout elements
    - Division of the section when guiding readers through code blocks or diagrams
    - Formatting of notes and code blocks
    - Use of bolding, italicizing, quotes, backticks, or other formatting elements
    - Formatting of citation references across sentences
    - Formatting of images, tables, and Mermaid diagrams and their corresponding citations. If they are missing from
    the generated section, we consider it valid for this criterion, as we are interested ONLY in formatting, which we
    cannot verify when elements are absent. For this criterion, missing elements from the generated sections are 
    considered valid. They are invalid only if present in both sections but formatted differently.
    - Number formatting conventions
9. Along with the binary scores, you will provide a brief and concise explanation containing the reasoning behind 
the score for each criterion. The score will be used to debug and monitor the evaluation process. Therefore, it is
important to provide thorough reasoning for the score. Since we provide binary scores, the reasoning should always 
contain what is good and what is problematic about the generated section, regardless of the score. For example, if the 
score is 0, the reasoning should also contain what is good about the generated section, such as "both sections 
follow the same flow of ideas," and what is problematic, such as "the generated section contains an additional 
paragraph on AI Evals that is not present in the expected section."
10. Important rules when comparing the content of sections:
    - Focus on substance, not superficial formatting differences
    - When comparing **media**, you only care about the placement of the media, not the content of the media. 
    Since media can take many forms such as Mermaid diagrams, tables, images, or URLs, you will completely ignore the 
    content of the media and only check whether the media is present in the correct place in the section, has 
    the appropriate citation, and proper numbering.
...
"""
```

**3. Chain of Thought** - Helping the LLM reason across all the intructions:
```python
"""
...
## CHAIN OF THOUGHT

**Understanding Input:**
1.1. Read, understand, and compare each section of the expected output and generated output.
1.2. Since we want to compute scores for each section of the expected output Markdown file, split the expected output 
into sections using the H2 headers as separators.

**Splitting into Sections:**
2.1. Using the expected output as the reference point, compare each section of the expected and generated 
outputs individually and assign a binary score of 0 or 1, where 0 indicates a mismatch and 1 indicates a perfect match.
2.2. Always use the expected output as the reference point to extract the sections of interest. 
2.3. When computing the score for an individual section, you will iterate through each section of the expected output, 
find its associated section in the generated output, and compute the score in isolation, ignoring all other sections.

**Assigning Scores to Each Section:**
3.1. Based on all sections of the expected output, assign a binary score of either 0 or 1 
for all evaluation criteria listed in the instructions:
    - **1:** The generated section matches the expected section perfectly on the given criterion.
    - **0:** The generated section does not match the expected section on the given criterion.
3.2. Justify why you assigned a score of 0 or 1 with a brief explanation that highlights the reasoning behind the score
based on the given criterion.
...
"""
```

**4. What to Avoid** - as a good practice to highlight key reasoning steps:
```python
"""
...
## WHAT TO AVOID

- Do not provide scores using the generated output as the reference point to divide into sections. You must always 
use the expected output as the reference point to divide into sections.
- Do not let other sections influence the score of a section. The score of each section must be determined in complete 
isolation from any other section.
- Do not overlap requirements between different criteria. For example, in the content criterion, as are not interested in the flow 
of ideas, if the ideas are in different order, or something is missing or additional, it's still valid. However, that is an important
aspect of the flow criterion, which will be invalid.
...
"""
```

**5. Few-shot examples and input placeholders:**
```python
"""
...
## FEW-SHOT EXAMPLES

Here are few-shot examples demonstrating how to compute the scores for each section and criterion:
<few-shot-examples>
{examples}
</few-shot-examples>

## INPUTS

<generated_output>
{output}
</generated_output>

<expected_output>
{expected_output}
</expected_output>
...
"""
```

**6. Final anchoring to the requested task:**
```python
"""
...
Think through your answer step by step, and provide the requested evaluation.
"""
```


### 4.8 Plugging the Few-Shot Examples Into the System Prompt

Now that we looked over how we model the few-shot examples entities and understood the system prompt, it's time to glue them together.

To do that, we have to define the few-shot examples that we manually created by:
1. Generating articles based on the article guideline and research inputs.
2. Adding noise to the generated article to create variation at the section level. Remember that in reality each section acts as it's own independent example. 
3. Manually labeling each section along each metric dimension.

Here's how we strctured the few-shot examples using the `FollowsGTMetricFewShotExamples`, `FollowsGTMetricFewShotExample` and `FollowsGTArticleScores` entity classes.

Source: `brown.evals.metrics.follows_gt.prompts`

```python
EXAMPLES_DIR = Path(__file__).parent / "examples"
EXAMPLES_DIR = Path(__file__).parent / "examples"
DEFAULT_FEW_SHOT_EXAMPLES = FollowsGTMetricFewShotExamples(
    examples=[
        FollowsGTMetricFewShotExample.from_markdown(
            output_file=EXAMPLES_DIR / "04_structured_outputs" / "article_generated.md",
            expected_output_file=EXAMPLES_DIR / "04_structured_outputs" / "article_ground_truth.md",
            scores=FollowsGTArticleScores(
                sections=[
                    FollowsGTSectionScores(
                        title="Introduction",
                        scores=FollowsGTCriterionScores(
                            content=CriterionScore(
                                score=1,
                                reason=(
                                    "Both sections cover the same core subjects and ideas, discussing the purpose "
                                    "of structured outputs as a bridge between LLMs and traditional applications."
                                ),
                            ),
                            flow=CriterionScore(
                                score=0,
                                reason=(
                                    "The generated section lacks the first sentence, which is used as a smooth "
                                    "transition into the article. Also, it misses the diagram present in the "
                                    "expected output, labeled as Figure 1."
                                ),
                            ),
                            structure=CriterionScore(
                                score=1,
                                reason="Both sections use the same paragraph length patterns.",
                            ),
                        ),
                    ),
                    FollowsGTSectionScores(
                        title="Why Structured Outputs Are Critical",
                        scores=FollowsGTCriterionScores(
                            content=CriterionScore(
                                score=0,
                                reason=(
                                    "Both sections cover the same reasons why structured outputs are critical, "
                                    "including ease of parsing, data validation with Pydantic, and common use "
                                    "cases. Still, the generated section has a section on GraphRAG, which is not "
                                    "related to the specific topic of the section."
                                ),
                            ),
                            flow=CriterionScore(
                                score=0,
                                reason=(
                                    "Follows a similar logical flow, starting with the importance, detailing "
                                    "benefits, discussing use cases, and concluding with a diagram. Still, the "
                                    "generated section contains an additional paragraph on GraphRAG, which "
                                    "doesn't fit with the expected flow. Additionally, it omits the last sentence, "
                                    "which is necessary for a smooth transition to the next section."
                                ),
                            ),
                            structure=CriterionScore(
                                score=1,
                                reason=(
                                    "Both sections use the same paragraph length patterns and have the same usage "
                                    "pattern for backticks and citation references across sentences. Also, the figures "
                                    "and their corresponding citations use the same formatting rules."
                                ),
                            ),
                        ),
                    ),
                    FollowsGTSectionScores(
                        title="Implementing Structured Outputs From Scratch Using JSON",
                        scores=FollowsGTCriterionScores(
                            content=CriterionScore(
                                score=1,
                                reason=(
                                    "Both sections provide a step-by-step guide on implementing structured outputs "
                                    "using JSON from scratch, covering client setup, document definition, prompt "
                                    "crafting, and parsing."
                                ),
                            ),
                            flow=CriterionScore(
                                score=0,
                                reason="The generated section omits the Note callout box present in the expected output.",
                            ),
                            structure=CriterionScore(
                                score=0,
                                reason=(
                                    "The generated section incorrectly formats the JSON code block under point 4), "
                                    "where it misses the closing ```. Also, in the last section, where it outputs "
                                    "the final JSON structure, it doesn't enclose the JSON into Python backticks "
                                    "as expected: ```python <content> ```"
                                ),
                            ),
                        ),
                    ),
                    ... # The rest of the sections 
                    FollowsGTSectionScores(
                        title="Structured Outputs Are Everywhere",
                        scores=FollowsGTCriterionScores(
                            content=CriterionScore(
                                score=0,
                                reason=(
                                    "Both sections serve as a conclusion, summarizing the importance of structured "
                                    "outputs as a fundamental pattern. Still, the generated section misses the last "
                                    "paragraph that presents how structured outputs fit in the course and the AI "
                                    "Engineering field, which is critical for the conclusion."
                                ),
                            ),
                            flow=CriterionScore(
                                score=0,
                                reason=(
                                    "Both sections follow a similar flow, summarizing the key takeaway. Still, the "
                                    "generated section misses the last paragraph on looking ahead to future lessons "
                                    "in the course."
                                ),
                            ),
                            structure=CriterionScore(
                                score=0,
                                reason=(
                                    "Both sections use the same paragraph length patterns. Still, the number "
                                    "formatting of the citation reference from the first paragraph misses the "
                                    "square brackets."
                                ),
                            ),
                        ),
                    ),
                    FollowsGTSectionScores(
                        title="References",
                        scores=FollowsGTCriterionScores(
                            content=CriterionScore(
                                score=1,
                                reason="Both sections contain a list of references, similar in purpose.",
                            ),
                            flow=CriterionScore(
                                score=1,
                                reason="Both sections follow the same flow for referencing the sources, as a numbered list from 1 to n.",
                            ),
                            structure=CriterionScore(
                                score=1,
                                reason=(
                                    "Both sections use the same pattern to structure the references, as a bulleted "
                                    "list, where each element is structured as [<reference_number>] "
                                    "[<reference_name>](<reference_url>)"
                                ),
                            ),
                        ),
                    ),
                ]
            ),
        ),
        FollowsGTMetricFewShotExample.from_markdown(
            output_file=EXAMPLES_DIR / "07_reasoning_planning" / "article_generated.md",
            expected_output_file=EXAMPLES_DIR / "07_reasoning_planning" / "article_ground_truth.md",
            scores=FollowsGTArticleScores(
                sections=[
                    FollowsGTSectionScores(
                        title="Introduction",
                        scores=FollowsGTCriterionScores(
                            content=CriterionScore(
                                score=1,
                                reason=(
                                    "Covers the same core subjects and ideas, discussing the limitations of standard "
                                    "LLMs and the need for planning and reasoning in AI agents."
                                ),
                            ),
                            flow=CriterionScore(
                                score=0,
                                reason=(
                                    "Both sections set the scene of the lesson, dicussing the 'why' behind the need for planning and "
                                    "reasoning in AI agents. However, the generated introduction omits the sentences that talk about "
                                    "the previous lessons and anchor the lesson within the course."
                                ),
                            ),
                            structure=CriterionScore(
                                score=0,
                                reason=(
                                    "The generated output uses an H2 header 'Why Your Agent Needs to Think Before It Acts' "
                                    "as a title for the introduction, while the expected section doesn't."
                                ),
                            ),
                        ),
                    ),
                    FollowsGTSectionScores(
                        title="What a Non-Reasoning Model Does And Why It Fails on Complex Tasks",
                        scores=FollowsGTCriterionScores(
                            content=CriterionScore(
                                score=1,
                                reason=(
                                    "Accurately covers the core subject of why non-reasoning models fail on complex "
                                    "tasks, using the same 'Technical Research Assistant Agent' example and discussing "
                                    "similar failure points."
                                ),
                            ),
                            flow=CriterionScore(
                                score=1,
                                reason=(
                                    "Follows a similar order of ideas, starting with the example, explaining the failure, "
                                    "and then discussing the need for reasoning."
                                ),
                            ),
                            structure=CriterionScore(
                                score=1,
                                reason=(
                                    "Both sections have similar paragraph length patterns and use of images and their "
                                    "corresponding citations."
                                ),
                            ),
                        ),
                    ),
                    ... # The rest of the sections
                    FollowsGTSectionScores(
                        title="ReAct in Depth: The Loop of Thought, Action, and Observation",
                        scores=FollowsGTCriterionScores(
                            content=CriterionScore(
                                score=1,
                                reason=(
                                    "The two sections provide the same detailed explanation of the ReAct framework, "
                                    "its iterative loop, and a step-by-step example using the research assistant agent."
                                ),
                            ),
                            flow=CriterionScore(
                                score=0,
                                reason=(
                                    "Both sections begin with the same flow, introducing ReAct, explaining its loop "
                                    "and presenting the diagram. The generated section has some additional reference numbers, which "
                                    "is correct. Still, the generated sections wrote the primary advantages and disadvantages of "
                                    "ReAct section before the hands-on example, instead of after it, as expected."
                                ),
                            ),
                            structure=CriterionScore(
                                score=0,
                                reason=(
                                    "The generated section employs a similar strategy to format the diagram's "
                                    "citation, references. However, in the expected section, the list is formatted as a "
                                    "numbered list, while in the generated section, it's formatted as a bulleted list. Also, the "
                                    "generated section added backquotes around the text from Action 1, 2, 3, and 4, while the "
                                    "expected section does not."
                                ),
                            ),
                        ),
                    ),
                    FollowsGTSectionScores(
                        title="Plan-and-Execute in Depth: Structure and Predictability",
                        scores=FollowsGTCriterionScores(
                            content=CriterionScore(
                                score=1,
                                reason=(
                                    "Accurately explains the Plan-and-Execute pattern, its two phases "
                                    "(Planning and Execution), and its benefits for predictable tasks."
                                ),
                            ),
                            flow=CriterionScore(
                                score=0,
                                reason=(
                                    "Both sections follow a similar logical flow, introducing the pattern, "
                                    "explaining its efficiency and then detailing the planning and execution "
                                    "phases with an example. The issue is that the Plan-and-Execute pattern "
                                    "diagram was expected before digging into the **Planning Phase** section, and "
                                    "instead, it's placed within the numbered list of the **Planning Phase** section."
                                ),
                            ),
                            structure=CriterionScore(
                                score=0,
                                reason=(
                                    "The generated section employs a similar strategy to format the diagram's "
                                    "citation, number formatting, references and the bulleted list. Still, it formats the planning "
                                    "and execution phases as bolded text instead of as H3 headers."
                                ),
                            ),
                        ),
                    ),
                    ... # The rest of the sections
                    FollowsGTSectionScores(
                        title='Reasoning Models: How LLMs\' "Reasoning and Planning" are Being Internalized in LLMs',
                        scores=FollowsGTCriterionScores(
                            content=CriterionScore(
                                score=0,
                                reason="The generated section is completely empty.",
                            ),
                            flow=CriterionScore(
                                score=0,
                                reason="The generated section is completely empty.",
                            ),
                            structure=CriterionScore(
                                score=0,
                                reason="The generated section is completely empty.",
                            ),
                        ),
                    ),
                    FollowsGTSectionScores(
                        title="Conclusion",
                        scores=FollowsGTCriterionScores(
                            content=CriterionScore(
                                score=1,
                                reason=(
                                    "In both sections, the conclusion summarizes the key takeaways of the article, "
                                    "including the importance of planning and reasoning, and the two foundational "
                                    "patterns (ReAct and Plan-and-Execute)."
                                ),
                            ),
                            flow=CriterionScore(
                                score=1,
                                reason=(
                                    "Follows a similar flow, reiterating the main points within the lesson "
                                    "and setting the scene for future lessons."
                                ),
                            ),
                            structure=CriterionScore(
                                score=1,
                                reason="Both sections have similar paragraph length, number formatting, and citation patterns.",
                            ),
                        ),
                    ),
                    FollowsGTSectionScores(
                        title="References",
                        scores=FollowsGTCriterionScores(
                            content=CriterionScore(
                                score=1,
                                reason="Both sections contain a list of citations, similar in purpose.",
                            ),
                            flow=CriterionScore(
                                score=1,
                                reason="Both sections follow the same flow for referencing the sources, as a numbered list from 1 to n.",
                            ),
                            structure=CriterionScore(
                                score=0,
                                reason=(
                                    "Both sections use a bulleted list to enumerate the citations, "
                                    "but the use of parentheses is not the same. The generated article outputs the references as "
                                    "`- [<number>] <reference_name>(<url>), ` instead of `- [[<number>]](<url>) <article_name>`."
                                ),
                            ),
                        ),
                    ),
                ]
            ),
        ),
    ]
)
```

Let's look at more details about the few-shot examples:


In [10]:
from brown.evals.metrics.follows_gt.prompts import DEFAULT_FEW_SHOT_EXAMPLES as FOLLOWS_GT_FEW_SHOT_EXAMPLES

print(f"Number of few-shot examples: {len(FOLLOWS_GT_FEW_SHOT_EXAMPLES.examples)}")
for i, example in enumerate(FOLLOWS_GT_FEW_SHOT_EXAMPLES.examples):
    print(f"\nExample {i + 1}:")
    print(f"  - Output length: {len(example.output):,} characters")
    print(f"  - Expected output length: {len(example.expected_output):,} characters")
    print(f"  - Number of sections scored: {len(example.scores.sections)}")
    print(f"  - Section titles: {[s.title for s in example.scores.sections]}")


Number of few-shot examples: 2

Example 1:
  - Output length: 20,182 characters
  - Expected output length: 24,788 characters
  - Number of sections scored: 7
  - Section titles: ['Introduction', 'Why Structured Outputs Are Critical', 'Implementing Structured Outputs From Scratch Using JSON', 'Implementing Structured Outputs From Scratch Using Pydantic', 'Implementing Structured Outputs Using Gemini and Pydantic', 'Structured Outputs Are Everywhere', 'References']

Example 2:
  - Output length: 26,000 characters
  - Expected output length: 33,942 characters
  - Number of sections scored: 11
  - Section titles: ['Introduction', 'What a Non-Reasoning Model Does And Why It Fails on Complex Tasks', 'Teaching Models to "Think": Chain-of-Thought and Its Limits', 'Separating Planning from Answering: Foundations of ReAct and Plan-and-Execute', 'ReAct in Depth: The Loop of Thought, Action, and Observation', 'Plan-and-Execute in Depth: Structure and Predictability', 'Pros and Cons: ReAct vs. Pla

In case you want to look at the full definition of the few-shot examples go to the [brown.evals.metrics.follows_gt.prompts.py](../writing_workflow/src/brown/evals/metrics/follows_gt/prompts.py) file from the Brown project.

### 4.9 The get_eval_prompt Function

To actually use the system prompt from above, together with the few-shot examples we defined, we defined a `get_eval_prompt` function that assembles the final prompt. This function is used within the `FollowsGTMetricLLMJudge` metric class we defined at the beginning, right before calling the LLM:

Source: `brown.evals.metrics.follows_gt.prompts`

```python
def get_eval_prompt(
    output: str,
    expected_output: str,
    few_shot_examples: FollowsGTMetricFewShotExamples,
) -> str:
    """Generate the evaluation prompt for the follows_gt metric."""

    return SYSTEM_PROMPT.format(
        examples=few_shot_examples.to_context(),
        output=output,
        expected_output=expected_output,
    )
```

This function takes the generated output, expected output, and few-shot examples, and formats them into the complete system prompt for the LLM judge.

Here is how the system prompt looks like:


In [11]:
from brown.evals.metrics.follows_gt.prompts import DEFAULT_FEW_SHOT_EXAMPLES, get_eval_prompt

# Crop the few-shot examples to 1 example and cropped dummy output and expected output to
# easily see the prompt structure
cropped_few_shot_examples = DEFAULT_FEW_SHOT_EXAMPLES.model_copy(deep=True)
cropped_few_shot_examples.examples = cropped_few_shot_examples.examples[:1]
cropped_few_shot_examples.examples[0].output = f"{cropped_few_shot_examples.examples[0].output[:1000]}..."
cropped_few_shot_examples.examples[
    0
].expected_output = f"{cropped_few_shot_examples.examples[0].expected_output[:1000]}..."

pretty_print.wrapped(
    get_eval_prompt(
        output="Dummy output article",
        expected_output="Dummy expected output article",
        few_shot_examples=cropped_few_shot_examples,
    ),
    title="FollowsGT LLM Judge System Prompt",
)

-------------------------------- FollowsGT LLM Judge System Prompt --------------------------------
   You are an expert in Natural Language Processing (NLP) evaluation metrics, specifically trained to 
assess answer quality in responses provided by large language models (LLMs). 

Your task is to evaluate the quality of a generated article by another LLM relative to 
an expected article output across multiple criteria: content, flow, structure, and mechanics.

## INSTRUCTIONS 

1. You must analyze the given expected article (<expected_output>) and generated article (<generated_output>) 
to determine the most relevant evaluation.
2. Since the generated output is an answer from another LLM, you must use the expected output as the reference 
standard to compare and evaluate the quality of the generated output.
3. Both the generated and expected outputs are in Markdown format.
4. Instead of comparing the outputs as a whole, you will divide the outputs into sections and compare each section

And this is how the few-shot examples data structure looks like when we map it to context:

In [12]:
pretty_print.wrapped(cropped_few_shot_examples.to_context(), title="FollowsGT LLM Judge Few-Shot Examples")

------------------------------ FollowsGT LLM Judge Few-Shot Examples ------------------------------
  <example_1>
	
<output>
# Lesson 4: Structured Outputs

We will start with a critical component for building production-grade AI systems: structured outputs. This concept is the bridge between the probabilistic, flexible world of Large Language Models (LLMs)—what some call Software 3.0—and the deterministic, rigid world of traditional Python code, or Software 1.0.

LLMs generate text, but our applications need data—objects, lists, and validated fields. Structured outputs provide a reliable contract that forces the model’s free-form text into a predictable format. Mastering this is not just a nice-to-have, it is essential for any AI Engineer who wants to build systems that are reliable, testable, and maintainable. Without it, you are just building fragile applications on a foundation of hope.

## Why Structured Outputs Are Critical

Before we write any code, it is important to understand

## 5. Running the FollowsGT Metric

Let's run the `FollowsGTMetricLLMJudge` on a sample to see it in action. We'll use the test sample from `inputs/tests/01_sample_small` as our expected output and create a variant with some modifications as the generated output.


In [13]:
EXPECTED_ARTICLE = (SAMPLE_DIR / "article.md").read_text()

pretty_print.wrapped(f"Expected article length: {len(EXPECTED_ARTICLE):,} characters", title="Expected Article")
pretty_print.wrapped(EXPECTED_ARTICLE, title="Expected Article Preview")


----------------------------------------- Expected Article -----------------------------------------
  Expected article length: 6,982 characters
----------------------------------------------------------------------------------------------------
------------------------------------- Expected Article Preview -------------------------------------
  # Workflows vs. Agents: The Critical Decision Every AI Engineer Faces
### How to choose between predictable control and autonomous flexibility when building AI applications.

When building AI applications, engineers face a critical architectural decision early on. Should you create a predictable, step-by-step workflow where you control every action, or build an autonomous agent that can think and decide for itself? This choice impacts everything from development time and cost to reliability and user experience. It is a fundamental decision that often determines if an AI application will be successful in production.

By the end of this lesson, 

And here is a noisy version of the same article where we deleted or changed parts of the article from above to trigger the LLM judge metrics. This will serve as our generated output from the Brown workflow:

In [14]:
OUTPUT_ARTICLE = (SAMPLE_DIR / "article_noisy.md").read_text()

pretty_print.wrapped(f"Generated article length: {len(OUTPUT_ARTICLE):,} characters", title="Generated Article")
pretty_print.wrapped(OUTPUT_ARTICLE, title="Generated Article Preview")


---------------------------------------- Generated Article ----------------------------------------
  Generated article length: 6,024 characters
----------------------------------------------------------------------------------------------------
------------------------------------ Generated Article Preview ------------------------------------
  # Workflows vs. Agents: The Critical Decision Every AI Engineer Faces
### How to choose between predictable control and autonomous flexibility when building AI applications.

By the end of this lesson, you will understand the fundamental differences between LLM workflows and AI agents, know when to use each, and recognize how to combine their strengths in hybrid approaches.

When building AI applications, engineers face a critical architectural decision early on. Should you create a predictable, step-by-step workflow where you control every action, or build an autonomous agent that can think and decide for itself? This choice impacts everything

In this noise version of the article, we:
- Changed the order of the paragraphs in the introduction.
- Deleted most of the paragraph from the `### LLM Workflows` section.
- Added incorrect information at the beginning of the `## Choosing Your Path` section.
- Mapped the `Choosing Your Path` section from using `###` sections to using a bullet list.
- Deleted the last paragraph from the conclusion.

Let's see how our LLM judge can detect these changes.

In [15]:
from brown.evals.metrics import FollowsGTMetricLLMJudge
from brown.models import SupportedModels

follows_gt_metric = FollowsGTMetricLLMJudge(
    model=SupportedModels.GOOGLE_GEMINI_25_FLASH,
    track=False,  # Disable tracking for this demo
)

print("Running FollowsGT metric...")
print("This may take a minute...")

results = await follows_gt_metric.ascore(
    output=OUTPUT_ARTICLE,
    expected_output=EXPECTED_ARTICLE,
)

print(f"\n✓ Generated {len(results)} score results")


Running FollowsGT metric...
This may take a minute...

✓ Generated 3 score results


Display the results:

In [16]:
for result in results:
    pretty_print.wrapped(f"{result.name} - Score: {result.value:.2f}", title=result.name)
    print("*** REASON ***\n")
    print(f"{result.reason[:500]}..." if len(result.reason) > 500 else result.reason)
    print("\n")

---------------------------------------- follows_gt_content ----------------------------------------
  follows_gt_content - Score: 0.40
----------------------------------------------------------------------------------------------------
*** REASON ***

Introduction:
**1:** Both sections introduce the core dilemma of choosing between LLM workflows and AI agents, and outline the learning objectives of the lesson.

Understanding the Spectrum: From Workflows to Agents:
**0:** The generated section's 'LLM Workflows' sub-section is significantly less detailed than the expected output, omitting crucial descriptions about predictability, control, testing, and debugging.

Choosing Your Path:
**0:** The generated section incorrectly identifies the core ...


----------------------------------------- follows_gt_flow -----------------------------------------
  follows_gt_flow - Score: 0.20
----------------------------------------------------------------------------------------------------
*** REAS

Note that even if we have a final aggregate score within the `[0, 1]` interval within the `REASON` attribute, we can see the binary score and the reason behind each score for each section. The final aggregate score is the average of the binary scores in the `REASON` field.

Now, that we understood how the `FollowsGTMetricLLMJudge` metric works, let's look over the `UserIntentMetricLLMJudge` metric, which follows a similar strategy starting from how the metric class is defined to how we wrote the prompt and few-shot examples.

## 6. The UserIntent Metric: Checking Guideline Adherence and Research Anchoring

Now let's explore the `UserIntentMetricLLMJudge`, our second LLM judge metric. Since we already covered the base abstractions, we'll focus on the particularities of this metric.

### 6.1 The UserIntentMetricLLMJudge Implementation

Source: `brown.evals.metrics.user_intent.metric`

```python
class UserIntentMetricLLMJudge(BrownBaseMetric):
    """A metric that evaluates how well generated articles follow article guidelines and are anchored in research."""

    def __init__(
        self,
        model: SupportedModels = SupportedModels.GOOGLE_GEMINI_25_FLASH,
        name: str = "user_intent",
        model_config: ModelConfig | None = None,
        track: bool = True,
        project_name: str | None = None,
    ) -> None:
        model_config = model_config or ModelConfig(
            temperature=0.0, thinking_budget=1024 * 4, include_thoughts=False, max_retries=3
        )
        super().__init__(
            model=model,
            name=name,
            structured_output_type=UserIntentArticleScores,
            few_shot_examples=prompts.DEFAULT_FEW_SHOT_EXAMPLES,
            model_config=model_config,
            track=track,
            project_name=project_name,
        )

    async def ascore(
        self,
        input: str,
        context: dict[str, Any],
        output: str,
        **ignored_kwargs: Any,
    ) -> score_result.ScoreResult | list[score_result.ScoreResult]:
        # Extract research from context
        if "research" not in context:
            raise ValueError("Context must contain a 'research' key with research content")

        research_content = context["research"]
        model_client = self.init_model()

        llm_query = prompts.get_eval_prompt(
            input=input,
            context=research_content,
            output=output,
            few_shot_examples=self.few_shot_examples,
        )

        user_intent_response = cast(
            UserIntentArticleScores,
            await model_client.ainvoke([{"role": "user", "content": llm_query}]),
        )

        if not user_intent_response:
            raise ValueError("Model failed to return a structured response.")

        return user_intent_response.to_score_result(self.name)
```

**Key Differences from FollowsGT:**
- Takes `input` (article guideline), `context` (with research), and `output` (generated article)
- Uses `UserIntentArticleScores` as the structured output type
- Evaluates two dimensions: `guideline_adherence` and `research_anchoring`


### 6.2 The Score Types

Source: `brown.evals.metrics.user_intent.types`

```python
class UserIntentCriteriaScores(CriteriaScores):
    """Represents scores for both evaluation dimensions of a section."""

    guideline_adherence: CriterionScore
    research_anchoring: CriterionScore


class UserIntentSectionScores(pydantic.BaseModel):
    """Section evaluation with scores across all UserIntent dimensions."""

    title: str = pydantic.Field(description="The title of the section being evaluated.")
    scores: UserIntentCriteriaScores = pydantic.Field(description="The scores of the section.")


class UserIntentArticleScores(pydantic.BaseModel):
    """Article-level scores for the UserIntent evaluation metric."""

    sections: list[UserIntentSectionScores]

    def to_score_result(self, prefix: str) -> list[score_result.ScoreResult]:
        return aggregate_section_scores_to_results(self.sections, prefix)
```

The structure mirrors `FollowsGT`, but with different dimensions:
- **guideline_adherence**: Does the section follow the article guideline?
- **research_anchoring**: Is the content based on the provided research?


### 6.3 The Few-Shot Examples

Source: `brown.evals.metrics.user_intent.types`

```python
class UserIntentMetricFewShotExample(BaseExample):
    """Represents a single example for the user intent evaluation."""

    input: str  # Article guideline
    context: str  # Research
    output: str  # Generated article
    scores: UserIntentArticleScores

    @classmethod
    def from_markdown(
        cls,
        input_file: Path,
        context_file: Path,
        output_file: Path,
        scores: UserIntentArticleScores,
    ) -> "UserIntentMetricFewShotExample":
        input_content = input_file.read_text()
        context_content = context_file.read_text()
        output_content = output_file.read_text()
        return cls(input=input_content, context=context_content, output=output_content, scores=scores)

    def to_context(self) -> str:
        return f"""
<input>
{self.input}
</input>
<context>
{self.context}
</context>
<output>
{self.output}
</output>
{self.scores.to_context()}
"""


class UserIntentMetricFewShotExamples(BaseFewShotExamples[UserIntentMetricFewShotExample]):
    """Collection of few-shot examples for the UserIntent evaluation metric."""
    pass
```

As you can see, the metric and entity classes are almost identical with the ones for the follows GT metric, but scoped on different metrics and inputs. Now, let's also look over the system prompt and concrete few-shot examples.

### 6.4 The System Prompt

The UserIntent prompt focuses on guideline adherence and research anchoring:

Source: `brown.evals.metrics.user_intent.prompts`

**1. Introduction:**
```python
SYSTEM_PROMPT = """
You are an expert in Natural Language Processing (NLP) evaluation metrics, specifically trained to 
assess how well generated content follows specific guidelines and incorporates provided research material. 

The AI system you will evaluate takes as input an article guideline (<input>) and research material (<context>), and 
based on these inputs, another large language model (LLM) generates an article (<output>).

Your task is to evaluate whether the generated article (<output>) adheres to the given
user intent structured as an article guideline and is properly anchored in the provided research.
...
"""
```

**2. Instructions - Defining the scoring business logic for the guideline adherence and research anchoring binary metrics:**
```python
"""
...
## INSTRUCTIONS 

1. You must analyze the given article guideline (<input>), research material (<context>), and 
generated article (<output>) to determine how well the output follows the input guidelines and utilizes the research.
2. The input, context, and output are in Markdown format.
3. Instead of comparing the outputs as a whole, you will divide the outputs into sections and compare each section 
individually. 
4. Since the article guideline reflects the user expectations and intent, you will always use it as the reference point 
to understand which sections from the generated article should be evaluated. To find the expected sections, look within 
the given article guideline for the keyword "Outline" or actual section titles often marked with H2 headers prefixed
with "Section". You will always use these as the anchor points, the expected sections, when making comparisons between
sections. 
5. When associating a section from the article guideline with one from the generated article, you will first look
for a matching or similar title. If there is no match based solely on the title, you will try to make associations based
on the content of the section. For example, if the expected section has 3 points on how RAG works and
the generated section has 3 points on how RAG works as well, then they are associated even if the titles are different.
If a required section mentioned in the guideline is missing from the generated article, you will assign a score of 
0 to all evaluation criteria.
6. Using the article guideline as an anchor, you will divide the generated article into sections and evaluate each section 
individually against all given criteria.
7. Sections are divided by H2 headers, marked as "##" in Markdown. You will use the headers as separators. 
Anything between two H2 headers constitutes a section. The only valid exception to this rule is the first section, 
the introduction, which sometimes appears between the title and the first H2 header. You will never include the title or 
subtitle as part of the first section.
8. The score can only be an integer value of 0 or 1. For each section, you will assign a binary integer score (0 or 1) based on 
two criteria:
   1. **Guideline Adherence**: For each expected section in the article guideline, you will evaluate whether the generated 
   section follows the specific section requirements outlined in the article guideline:
        - We expect a perfect match between the expected section and the generated section. Intuitively, you can
        think of the section guideline as a sketch, a compressed version of the generated section.
        - Less: If any topic from the expected article guideline is missing from the generated article, you will assign 
        a score of 0.
        - More: If the generated section has any additional topics that are not in the expected article guideline, 
        you will assign a score of 0.
        - Different Order: If the order of ideas from the expected article guideline is not followed in the 
        generated article, you will assign a score of 0.
        - If section constraints are provided, we are looking only for a rough approximation of the length. The exact
        section length criterias are present in the article guideline. Errors of ±100 units are acceptable. Units can
        be words, characters, or reading time. For example, if the expected section length is 100 words and the generated section length 
        is 190 words, you will assign a score of 1. But if the generated section is 230 words, as it exceeds the tolerance range,
        you will assign a score of 0.
   2. **Research Anchoring**: For each expected section in the article guideline, you will evaluate whether the generated 
   section content is based on or derived from the provided research:
        - We expect each section from the generated article to be generated entirely based on the ideas provided
        in the article guideline and research. Thus, you can consider both the context and article guideline as the 
        "research", the single source of truth.
        - If any idea from the generated section is not present in the research, you will assign a score of 0.
        - The generated section does not have to contain all the ideas from the research, just a subset of them.
        - If no research is explicitly referenced through citations, you will manually check if the generated section content
        it's based solely on the research. Missing explicit citations is valid. What it's critical is all the ideas to
        adhere to the research. Thus, if the generated section content is based solely on the research, while missing citations,
        you will assign a score of 1.
9. Along with the binary score, you will provide a brief and concise explanation containing the reasoning behind 
the score for each criterion. The score will be used to debug and monitor the evaluation process. Therefore, it is
important to provide thorough reasoning for the score. Since we provide binary scores, the reasoning should always 
contain what is good and what is problematic about the generated section, regardless of the score. For example, if the score 
is 0, the reasoning should also contain what is good about the generated section, such as "the generated section 
contains all the bulleted points from the expected section guideline," and what is problematic, such as "however, it contains an 
additional section on AI Evals that is not present in the guideline." Also, when generating the reasoning for the
research anchoring criterion, you will always mention if the topic comes from the article guideline, context, or both, while
supporting every single claim with evidence from the research. For example, the generated section is correctly anchored in the research,
while the fundamentals on RAG are based on the context, while the specific details on the RAG architecture are based on the article
guideline.
10. Important rules when evaluating:
   - Focus on substance, not superficial formatting differences
   - When comparing **media**, you only care about the placement and the caption of the media. 
    Since media can take many forms such as Mermaid diagrams, images, or URLs, you will completely ignore the 
    content of the media. Based on the section guideline, you will check whether the media is present in the 
    correct place. Based on the caption of the media, you will check whether it is properly anchored in the research.
...
"""
```

**3. Chain of Thought:**
```python
"""
...
## CHAIN OF THOUGHT

**Understanding Input:**
1.1. Read and understand the article guideline (<input>) to identify specific requirements, structure, 
content expectations, constraints, and most importantly the expected sections.
1.2. Read and understand the context (<context>) to identify available information, sources, and key findings.
1.3. Label the article guideline and context as the "research", the single source of truth.
1.4. Read and understand the generated article (<output>) and split it into sections using H2 headers as separators.
1.5. Connect the expected sections from the article guideline to the sections from the generated article.

**Section-by-Section Evaluation:**
2.1. For each section identified in the article guideline, locate its associated section in the generated article, and 
evaluate it against both criteria. If a section is found in the article guideline and is missing in the generated 
article, you will assign a score of 0 to all evaluation criteria.
2.2. Evaluate guideline adherence between each expected section from the article guideline and the associated section 
from the generated article.
2.3. Evaluate research anchoring by first selecting the sections to evaluate from the article guideline and then 
comparing the generated section to the research, found in the context and the associated section guideline.

**Assigning Scores:**
3.1. Based on each section expected from the article guideline, assign a binary score of either 0 or 1 
for all evaluation criteria listed in the instructions:
    - Score 1 if the section clearly follows the requirements detailed in the instructions.
    - Score 0 if it fails to follow the requirements detailed in the instructions.
3.2. Justify why you assigned a score of 0 or 1 with a brief explanation that highlights the reasoning behind the score
based on the given criterion.
...
"""
```

**4. What to Avoid:**
```python
"""
...
## WHAT TO AVOID

- Do not provide scores using the generated output as the reference point to divide into sections. You must always 
use the article guideline as the reference point to divide into sections.
- Do not let other sections influence the score of a section. The score of each section must be determined in complete 
isolation from any other section.
...
"""
```

**5. Few-shot examples and input:**
```python
"""
...
## FEW-SHOT EXAMPLES

Here are few-shot examples demonstrating how to compute the scores for each section and criterion:
<few-shot-examples>
{examples}
</few-shot-examples>

## INPUTS

<input>
{input}
</input>

<context>
{context}
</context>

<output>
{output}
</output>
...
"""
```

**6. Final anchoring to the requested task:**
```python
"""
...
Think through your answer step by step, and provide the requested evaluation.
"""


### 6.5 The Default Few-Shot Examples

Similar to FollowsGT, the few-shot examples are manually labeled and statically loaded within the `UserIntentMetricFewShotExamples`, `UserIntentMetricFewShotExample` and `UserIntentArticleScores` entity classes.

Notice how, for each section, for each metric type (guideline adherence and research anchoring), we manually added a binary score and the reason behind WHY we added that particular binary score as a 1-2 sentence explanation.

```python
EXAMPLES_DIR = Path(__file__).parent / "examples"
DEFAULT_FEW_SHOT_EXAMPLES = UserIntentMetricFewShotExamples(
    examples=[
        UserIntentMetricFewShotExample.from_markdown(
            input_file=EXAMPLES_DIR / "04_structured_outputs" / "article_guideline.md",
            context_file=EXAMPLES_DIR / "04_structured_outputs" / "research.md",
            output_file=EXAMPLES_DIR / "04_structured_outputs" / "article_generated.md",
            scores=UserIntentArticleScores(
                sections=[
                    UserIntentSectionScores(
                        title="Introduction",
                        scores=UserIntentCriteriaScores(
                            guideline_adherence=CriterionScore(
                                score=0,
                                reason=(
                                    'The section correctly introduces the "bridge" concept but violates three guidelines: '
                                    'it uses the first-person singular ("I remember...") against the point-of-view rule, '
                                    "adds a new anecdote about sentiment analysis and at ~250 words, it exceeds the "
                                    "150-word length constraint."
                                ),
                            ),
                            research_anchoring=CriterionScore(
                                score=0,
                                reason=(
                                    'While the core "bridge" concept between the LLM (software 3.0) and Python '
                                    "(software 1.0) worlds' is anchored in the guideline, the section introduces a "
                                    'significant, un-anchored claim that mastering structured outputs is "the key to '
                                    'unlocking true Artificial General Intelligence (AGI)," which is not supported by '
                                    "the research."
                                ),
                            ),
                        ),
                    ),
                    UserIntentSectionScores(
                        title="Understanding why structured outputs are critical",
                        scores=UserIntentCriteriaScores(
                            guideline_adherence=CriterionScore(
                                score=0,
                                reason=(
                                    "The section correctly covers the required benefits but violates the "
                                    '"Different Order" rule by presenting them in a different sequence than the '
                                    'guideline. It also adds an unrequested benefit ("Improved Token Efficiency"), '
                                    'violating the "More" rule.'
                                ),
                            ),
                            research_anchoring=CriterionScore(
                                score=1,
                                reason=(
                                    "The section is well-anchored. The benefits of parsing, Pydantic validation, "
                                    "avoiding fragile parsing and the GraphRAG/knowledge-graph use-case are all "
                                    "supported in the research and the added point on token efficiency are all "
                                    "directly supported by claims in the provided research material"
                                ),
                            ),
                        ),
                    ),
                    ... # The rest of the sections
                    UserIntentSectionScores(
                        title="Conclusion: Structured Outputs Are Everywhere",
                        scores=UserIntentCriteriaScores(
                            guideline_adherence=CriterionScore(
                                score=0,
                                reason=(
                                    "The section correctly emphasizes the ubiquity of structured outputs and "
                                    "name-checks later topics as the guideline asks for. However, it does not "
                                    "provide a transition to the requested next Lesson 5."
                                ),
                            ),
                            research_anchoring=CriterionScore(
                                score=1,
                                reason=(
                                    "The section's claims are general and consistent with the course context. "
                                    "It does not introduce new factual claims that require specific anchoring, "
                                    "and therefore contains no information that contradicts the provided research "
                                    "material."
                                ),
                            ),
                        ),
                    ),
                ]
            ),
        ),
        UserIntentMetricFewShotExample.from_markdown(
            input_file=EXAMPLES_DIR / "07_reasoning_planning" / "article_guideline.md",
            context_file=EXAMPLES_DIR / "07_reasoning_planning" / "research.md",
            output_file=EXAMPLES_DIR / "07_reasoning_planning" / "article_generated.md",
            scores=UserIntentArticleScores(
                sections=[
                    UserIntentSectionScores(
                        title="Introduction",
                        scores=UserIntentCriteriaScores(
                            guideline_adherence=CriterionScore(
                                score=1,
                                reason=(
                                    "The introduction aligns with the guideline. It sets the stage by introducing "
                                    "planning and reasoning, explains the problem with standard LLMs, previews ReAct "
                                    "and Plan-and-Execute, and mentions the advanced capabilities, all while staying "
                                    "within the specified word count."
                                ),
                            ),
                            research_anchoring=CriterionScore(
                                score=1,
                                reason=(
                                    "The section is fully anchored in the research. All the high-level concepts it "
                                    "introduces—planning, reasoning, ReAct, Plan-and-Execute, and self-correction—are "
                                    "the central themes of the provided research material and are presented accurately."
                                ),
                            ),
                        ),
                    ),
                    UserIntentSectionScores(
                        title="What a Non-Reasoning Model Does And Why It Fails on Complex Tasks",
                        scores=UserIntentCriteriaScores(
                            guideline_adherence=CriterionScore(
                                score=1,
                                reason=(
                                    "The section meets all requirements from the outline, including the correct agent "
                                    "example, consequences, and transitions. The addition of context on the 'black box "
                                    "problem' is brief and does not detract from or replace any of the mandated points."
                                ),
                            ),
                            research_anchoring=CriterionScore(
                                score=0,
                                reason=(
                                    "While the core concepts are based on the guideline, the section's added discussion "
                                    "on the 'black box problem' and its relation to 'enterprise adoption' is not "
                                    "supported by any evidence within the provided research material."
                                ),
                            ),
                        ),
                    ),
                    UserIntentSectionScores(
                        title='Teaching Models to "Think": Chain-of-Thought and Its Limits',
                        scores=UserIntentCriteriaScores(
                            guideline_adherence=CriterionScore(
                                score=0,
                                reason=(
                                    "The section uses the correct prompt example and most request topics but it fails "
                                    "to address the specific limitation required by the guideline, which is that the "
                                    "'plan and the answer appear in the same text'. Instead, it substitutes this with "
                                    "an unrequested point about computational cost."
                                ),
                            ),
                            research_anchoring=CriterionScore(
                                score=1,
                                reason=(
                                    "The section's description of Chain-of-Thought is correctly aligned with the "
                                    "foundational concepts presented in the research materials, such as the canonical "
                                    "paper Sourced."
                                ),
                            ),
                        ),
                    ),
                    ... # The rest of the sections
                    UserIntentSectionScores(
                        title="Plan-and-Execute in Depth: Structuring the Path from Goal to Action",
                        scores=UserIntentCriteriaScores(
                            guideline_adherence=CriterionScore(
                                score=0,
                                reason=(
                                    "Includes mermaid diagram and maps out the plan and execute example as requested. "
                                    "The section fails to adhere to the content guideline for its 'Pros'. It incorrectly "
                                    "emphasizes 'fostering model creativity' instead of the specified advantages of "
                                    "efficiency and reliability for structured tasks."
                                ),
                            ),
                            research_anchoring=CriterionScore(
                                score=0,
                                reason=(
                                    "A source for inflexibility is found correctly but the primary advantage "
                                    "presented—creativity—is not supported by the research set. The provided sources "
                                    "frame the benefits of this pattern in terms of cost, reliability, and efficiency."
                                ),
                            ),
                        ),
                    ),
                    ... # The rest of the sections
                    UserIntentSectionScores(
                        title="Conclusion",
                        scores=UserIntentCriteriaScores(
                            guideline_adherence=CriterionScore(
                                score=1,
                                reason=(
                                    "The conclusion adheres to the guideline. It effectively summarizes the lesson's "
                                    "key takeaways, recaps the ReAct and Plan-and-Execute patterns, and provides a "
                                    "clear, accurate preview of the next lessons (8, 9, and 10), all within the "
                                    "specified length"
                                ),
                            ),
                            research_anchoring=CriterionScore(
                                score=1,
                                reason=(
                                    "The section is fully anchored. The summary accurately recaps the core concepts "
                                    "supported by the research, and the preview of future lessons (8, 9, and 10) is "
                                    "taken directly from the article guideline."
                                ),
                            ),
                        ),
                    ),
                ]
            ),
        ),
    ]
)
```

Let's look at more details about the few-shot examples:


In [17]:
from brown.evals.metrics.user_intent.prompts import DEFAULT_FEW_SHOT_EXAMPLES as USER_INTENT_FEW_SHOT_EXAMPLES

print(f"Number of few-shot examples: {len(USER_INTENT_FEW_SHOT_EXAMPLES.examples)}")
for i, example in enumerate(USER_INTENT_FEW_SHOT_EXAMPLES.examples):
    print(f"\nExample {i + 1}:")
    print(f"  - Input (guideline) length: {len(example.input):,} characters")
    print(f"  - Context (research) length: {len(example.context):,} characters")
    print(f"  - Output (article) length: {len(example.output):,} characters")
    print(f"  - Number of sections scored: {len(example.scores.sections)}")
    print(f"  - Section titles: {[s.title for s in example.scores.sections]}")


Number of few-shot examples: 2

Example 1:
  - Input (guideline) length: 18,329 characters
  - Context (research) length: 385,985 characters
  - Output (article) length: 18,376 characters
  - Number of sections scored: 6
  - Section titles: ['Introduction', 'Understanding why structured outputs are critical', 'Implementing structured outputs from scratch using JSON', 'Implementing structured outputs from scratch using Pydantic', 'Implementing structured outputs using Gemini and Pydantic', 'Conclusion: Structured Outputs Are Everywhere']

Example 2:
  - Input (guideline) length: 20,085 characters
  - Context (research) length: 255,315 characters
  - Output (article) length: 19,238 characters
  - Number of sections scored: 10
  - Section titles: ['Introduction', 'What a Non-Reasoning Model Does And Why It Fails on Complex Tasks', 'Teaching Models to "Think": Chain-of-Thought and Its Limits', 'Separating Planning from Answering: Foundations of ReAct and Plan-and-Execute', 'ReAct in Depth:

### 6.6 The get_eval_prompt Function

To actually use the system prompt from above, together with the few-shot examples we defined, we defined a `get_eval_prompt` function that assembles the final prompt. This function is used within the `UserIntentMetricLLMJudge` metric class we defined at the beginning, right before calling the LLM:

Source: `brown.evals.metrics.user_intent.prompts`

```python
def get_eval_prompt(
    input: str,
    context: str,
    output: str,
    few_shot_examples: UserIntentMetricFewShotExamples,
) -> str:
    """Generate the evaluation prompt for the user intent metric."""

    return SYSTEM_PROMPT.format(
        examples=few_shot_examples.to_context(),
        input=input,
        context=context,
        output=output,
    )
```

This function takes the article guideline, research, generated output, and few-shot examples, and formats them into the complete system prompt for the LLM judge.

Here is how the system prompt looks like:


In [18]:
from brown.evals.metrics.user_intent.prompts import DEFAULT_FEW_SHOT_EXAMPLES, get_eval_prompt

# Crop the few-shot examples to 1 example and cropped dummy output and expected output to
# easily see the prompt structure
cropped_few_shot_examples = DEFAULT_FEW_SHOT_EXAMPLES.model_copy(deep=True)
cropped_few_shot_examples.examples = cropped_few_shot_examples.examples[:1]
cropped_few_shot_examples.examples[0].output = f"{cropped_few_shot_examples.examples[0].output[:1000]}..."
cropped_few_shot_examples.examples[0].input = f"{cropped_few_shot_examples.examples[0].input[:1000]}..."
cropped_few_shot_examples.examples[0].context = f"{cropped_few_shot_examples.examples[0].context[:1000]}..."

pretty_print.wrapped(
    get_eval_prompt(
        output="Dummy output article",
        input="Dummy input guideline",
        context="Dummy research",
        few_shot_examples=cropped_few_shot_examples,
    ),
    title="UserIntent LLM Judge System Prompt",
)

-------------------------------- UserIntent LLM Judge System Prompt --------------------------------
  You are an expert in Natural Language Processing (NLP) evaluation metrics, specifically trained to 
assess how well generated content follows specific guidelines and incorporates provided research material. 

The AI system you will evaluate takes as input an article guideline (<input>) and research material (<context>), and 
based on these inputs, another large language model (LLM) generates an article (<output>).

Your task is to evaluate whether the generated article (<output>) adheres to the given
user intent structured as an article guideline and is properly anchored in the provided research.

## INSTRUCTIONS 

1. You must analyze the given article guideline (<input>), research material (<context>), and 
generated article (<output>) to determine how well the output follows the input guidelines and utilizes the research.
2. The input, context, and output are in Markdown format.
3. 

And this is how the few-shot examples data structure looks like when we map it to context:

In [19]:
pretty_print.wrapped(cropped_few_shot_examples.to_context(), title="User Intent LLM Judge Few-Shot Examples")

----------------------------- User Intent LLM Judge Few-Shot Examples -----------------------------
  <example_1>
	
<input>
## Global Context of the Lesson

### What We Are Planning to Share

In this lesson, we will talk about structured outputs. We'll introduce **Structured Outputs** as a way to achieve reliable, controlled, and formatted data extraction, ensuring type and data quality checks. Common structured output types are JSON, XML, and YAML, which can further be translated into Python dictionaries, lists, classes, or most commonly Pydantic models. This approach makes manipulating LLM outputs much easier as it serves as the bridge between the LLM (software 3.0) and Python (software 1.0) worlds. First, we will implement structured outputs from scratch, then show how to do it directly with a popular API such as Gemini. Ultimately, we will highlight how structured outputs are used everywhere when building LLM workflows or AI agents.

### Why We Think It's Valuable

For an AI Engine

## 7. Running the UserIntent Metric

Let's run the `UserIntentMetricLLMJudge` on a sample. We'll use the same sample tests as for the follows GT metric.

Here is the output article as a refresher:


In [20]:
pretty_print.wrapped(OUTPUT_ARTICLE, title="Generated Article Preview")

------------------------------------ Generated Article Preview ------------------------------------
  # Workflows vs. Agents: The Critical Decision Every AI Engineer Faces
### How to choose between predictable control and autonomous flexibility when building AI applications.

By the end of this lesson, you will understand the fundamental differences between LLM workflows and AI agents, know when to use each, and recognize how to combine their strengths in hybrid approaches.

When building AI applications, engineers face a critical architectural decision early on. Should you create a predictable, step-by-step workflow where you control every action, or build an autonomous agent that can think and decide for itself? This choice impacts everything from development time and cost to reliability and user experience. It is a fundamental decision that often determines if an AI application will be successful in production.

## Understanding the Spectrum: From Workflows to Agents

To make the ri

The article guideline:

In [21]:
ARTICLE_GUIDELINE = (SAMPLE_DIR / "article_guideline.md").read_text()

pretty_print.wrapped(ARTICLE_GUIDELINE, title="Article Guideline")

---------------------------------------- Article Guideline ----------------------------------------
  ## Outline

1. Introduction: The Critical Decision Every AI Engineer Faces
2. Understanding the Spectrum: From Workflows to Agents
3. Choosing Your Path
4. Conclusion: The Challenges of Every AI Engineer

## Section 1 - Introduction: The Critical Decision Every AI Engineer Faces

- **The Problem:** When building AI applications, engineers face a critical architectural decision early in their development process. Should they create a predictable, step-by-step workflow where they control every action, or should they build an autonomous agent that can think and decide for itself? This is one of the key decisions that will impact everything from the product such as development time and costs to reliability and user experience.
- Quick walkthrough of what we'll learn by the end of this lesson

- **Section length:** 100 words

## Section 2 - Understanding the Spectrum: From Workflows to Agen

And the research:

In [22]:
RESEARCH = (SAMPLE_DIR / "research.md").read_text()

pretty_print.wrapped(f"{RESEARCH[:1000]}...", title="Research")


--------------------------------------------- Research ---------------------------------------------
  # Research

## Code Sources

<details>
<summary>Repository analysis for https://github.com/google-gemini/gemini-cli/blob/main/README.md</summary>

# Repository analysis for https://github.com/google-gemini/gemini-cli/blob/main/README.md

## Summary
Repository: google-gemini/gemini-cli
File: README.md
Lines: 211

Estimated tokens: 1.6k

## File tree
```Directory structure:
└── README.md

```

## Extracted content
FILE: README.md
# Gemini CLI

[![Gemini CLI CI](https://github.com/google-gemini/gemini-cli/actions/workflows/ci.yml/badge.svg)](https://github.com/google-gemini/gemini-cli/actions/workflows/ci.yml)

![Gemini CLI Screenshot](./docs/assets/gemini-screenshot.png)

This repository contains the Gemini CLI, a command-line AI workflow tool that connects to your
tools, understands your code and accelerates your workflows.

With the Gemini CLI you can:

- Query and edit large codebase

Finally, let's call the LLM judge:

In [23]:
from brown.evals.metrics import UserIntentMetricLLMJudge

user_intent_metric = UserIntentMetricLLMJudge(
    model=SupportedModels.GOOGLE_GEMINI_25_FLASH,
    track=False,  # Disable tracking for this demo
)

print("Running UserIntent metric...")
print("This may take a minute...")

results = await user_intent_metric.ascore(
    input=ARTICLE_GUIDELINE,
    context={"research": RESEARCH},
    output=OUTPUT_ARTICLE,
)

print(f"\n✓ Generated {len(results)} score results")


Running UserIntent metric...
This may take a minute...

✓ Generated 2 score results


Display the results:

In [24]:
for result in results:
    pretty_print.wrapped(f"{result.name} - Score: {result.value:.2f}", title=result.name)
    print("*** REASON ***\n")
    print(f"{result.reason[:1000]}..." if len(result.reason) > 1000 else result.reason)
    print("\n")

--------------------------------- user_intent_guideline_adherence ---------------------------------
  user_intent_guideline_adherence - Score: 0.50
----------------------------------------------------------------------------------------------------
*** REASON ***

Introduction: The Critical Decision Every AI Engineer Faces:
**1:** The section correctly introduces the problem of architectural decisions in AI, and provides a quick walkthrough of what will be learned. The length is approximately 90 words, which is within the 100-word limit (allowing for a 10% tolerance).

Understanding the Spectrum: From Workflows to Agents:
**1:** The section correctly defines LLM workflows and AI agents, focusing on their properties and usage rather than technical specifics. It includes both required Mermaid diagrams and adheres to the approximate 200-word length constraint (excluding diagrams).

Choosing Your Path:
**0:** The section fails to adhere to the guideline's instruction to explore the core di

## 8. The Metrics Factory Function

To simplify metric creation, we provide a factory function:

Source: `brown.evals.metrics.__init__`

```python
def build_evaluation_metrics(
    metrics: list[str], model: SupportedModels, model_config: ModelConfig | None = None
) -> list[base_metric.BaseMetric]:
    """Get evaluation metrics based on the provided metric names.

    Args:
        metrics: List of metric names to use. Valid values are:
            - "user_intent": Evaluates if article follows guidelines and is anchored in research
            - "follows_gt": Evaluates article structure and content
        model: The language model to use for evaluation.
        model_config: Configuration for the model.

    Returns:
        List of metric instances

    Raises:
        ValueError: If an unknown metric name is provided
    """
    metrics_mapping = {
        "user_intent": UserIntentMetricLLMJudge(model=model, model_config=model_config),
        "follows_gt": FollowsGTMetricLLMJudge(model=model, model_config=model_config),
    }

    try:
        return [metrics_mapping[metric] for metric in metrics]
    except KeyError as e:
        raise ValueError(f"Unknown metric name: {e}")
```


For example, let's build both metrics using the factory function:

In [25]:
from brown.evals.metrics import build_evaluation_metrics

all_metrics = build_evaluation_metrics(
    metrics=["user_intent", "follows_gt"],
    model=SupportedModels.GOOGLE_GEMINI_25_FLASH,
)

print("Created metrics:")
for metric in all_metrics:
    print(f"  - {metric.name}: {metric.__class__.__name__}")


Created metrics:
  - user_intent: UserIntentMetricLLMJudge
  - follows_gt: FollowsGTMetricLLMJudge


## 9. Running Evaluations on the Dataset

As we have to run the LLM judges on the whole AI evals dataset, we need a way to automate running Brown on multiple dataset samples on our business metrics.

We need several components working together.

### 9.1 The Evaluation Task

The `evaluation_task` function generates an article for a single dataset sample:

Source: `brown.evals.tasks`

```python
@a.as_sync
async def evaluation_task(
    sample: EvalSampleDict,
    cache_dir: Path,
    read_from_cache: bool = False,
    clean_cache: bool = False,
    debug: bool = False,
) -> Dict[str, Any]:
    """Generate an article using the Brown agent for a single evaluation sample."""
    logger.info(f"Processing evaluation sample: {sample['name']}")

    cache_dir = cache_dir / sample["directory"]
    cache_dir.mkdir(parents=True, exist_ok=True)

    try:
        article_guideline_path = cache_dir / "article_guideline.md"
        research_path = cache_dir / "research.md"

        article_guideline_path.write_text(sample["article_guideline"], encoding="utf-8")
        research_path.write_text(sample["research"], encoding="utf-8")

        thread_id = str(uuid.uuid4())
        inputs = {"dir_path": cache_dir}
        tracer = tracing.build_handler(thread_id, tags=["generate-evaluation"])
        config = RunnableConfig(
            configurable={"thread_id": thread_id},
            callbacks=[tracer],
        )

        try:
            generated_article = await __run(config, inputs, read_from_cache)
        except Exception:
            generated_article = "ERROR: Failed to generate article"
            logger.exception(f"Failed to generate article for sample `{sample['name']}`")

        return {
            "input": sample["article_guideline"],
            "context": {"research": sample["research"], "debug": debug},
            "output": generated_article,
            "expected_output": sample["ground_truth_article"],
            "reference": sample["ground_truth_article"],
            "name": sample["name"],
        }
    finally:
        if clean_cache and cache_dir.exists():
            shutil.rmtree(cache_dir)
```

**Key Points:**
- Takes an `EvalSampleDict` from the dataset
- Writes inputs to a cache directory
- Runs the Brown workflow to generate an article
- Returns a dictionary with all the data needed for scoring


### 9.2 The __run Function

The `__run` function handles the actual workflow execution:

Source: `brown.evals.tasks`

```python
async def __run(config: RunnableConfig, inputs: Dict[str, Any], read_from_cache: bool = False) -> str:
    """Run the article generation workflow and extract the final article."""

    app_config = get_app_config()

    article_path = inputs["dir_path"] / "article.md"
    if read_from_cache:
        assert article_path.exists(), f"Article file not found in cache at `{article_path}`"
        logger.success(f"Successfully read article from cache at `{article_path}`")
    else:
        async with build_short_term_memory(app_config) as checkpointer:
            generate_article_workflow = build_generate_article_workflow(checkpointer=checkpointer)
            await generate_article_workflow.ainvoke(inputs, config)

        logger.success(f"Successfully generated article at `{article_path}`")

    article = article_path.read_text(encoding="utf-8")

    return article
```

**Key Features:**
- Supports caching: Can read from cache or generate fresh
- Uses the same workflow as production Brown
- Returns the generated article as a string

💡 Note how easy it is to reuse our Brown code due to our clean architecture design.

## 10. Hooking to Opik for Full Evaluation

To run the full evaluation pipeline, we need glue code that connects our metrics to the `brown-course-lessons` dataset stored in Opik.

### 10.1 The evaluate Function

Source: `brown.observability.evaluation`

```python
from typing import Any, Callable, Literal

from loguru import logger
from opik import evaluation
from opik.evaluation.metrics import base_metric

from brown.config import get_settings
from brown.config_app import get_app_config as load_app_config

from .opik_utils import get_dataset

app_config = load_app_config()

def evaluate(
    dataset_name: str,
    metrics: list[base_metric.BaseMetric],
    evaluation_task: Callable,
    llm_judge_config: dict[str, Any],
    workers: int = 2,
    nb_samples: int | None = None,
    dataset_item_names: list[str] | None = None,
) -> None:
    """Run an Opik evaluation with the provided metrics and task."""

    assert get_settings().OPIK_API_KEY, "OPIK_API_KEY is not set."

    dataset = get_dataset(dataset_name)
    if not dataset:
        raise ValueError(f"Dataset '{dataset_name}' does not exist.")
    
    if dataset_item_names:
        all_dataset_items = dataset.get_items()
        dataset_item_ids = [item["id"] for item in all_dataset_items if item["name"] in dataset_item_names]
        logger.info(f"Evaluating {len(dataset_item_ids)}/{len(all_dataset_items)} dataset items.")
    else:
        dataset_item_ids = None

    logger.info("Starting evaluation...")

    llm_judge_config = {
        "dataset_name": dataset.name,
        "llm_judge_config": llm_judge_config,
        "app_config": app_config.model_dump(mode="json"),
    }

    evaluation.evaluate(
        dataset=dataset,
        task=evaluation_task,
        scoring_metrics=metrics,
        experiment_config=llm_judge_config,
        task_threads=workers,
        nb_samples=nb_samples,
        dataset_item_ids=dataset_item_ids,
    )
```

This function:
- Fetches the dataset from Opik
- Optionally filters to specific dataset items
- Runs the evaluation task on each sample
- Scores results with the provided metrics


### 10.2 Helper Functions

Source: `brown.observability.opik_utils`

```python
def get_dataset(name: str) -> opik.Dataset | None:
    """Get a dataset by name."""
    client = opik.Opik()
    try:
        dataset = client.get_dataset(name=name)
    except Exception:
        dataset = None
    return dataset
```

Source: `brown.evals.tasks`

```python
def create_evaluation_task(
    cache_dir: Path,
    read_from_cache: bool = False,
    clean_cache: bool = False,
    debug: bool = False,
) -> Callable:
    """Create a reusable evaluation task with fixed runtime parameters.
    
    We need this because Opik's evaluate function doesn't allow passing extra
    parameters to the task function. We use Python's `partial` to pin the
    cache_dir and other options.
    """
    return partial(
        evaluation_task,
        cache_dir=cache_dir,
        read_from_cache=read_from_cache,
        clean_cache=clean_cache,
        debug=debug,
    )
```


## 11. Running the End-to-End Evaluation

Now, let's glue everything together into the AI evals pipeline and run the end-to-end code we presented so far.

💡 You can run the exact same code using the `scripts/brown_run_eval.py` script from the `lessons/writing_workflow` Brown project and the `brown-run-eval-flash` / `brown-run-eval-pro` Make commands.

> ⚠️ **Warning**: Running the full evaluation generates articles for each dataset sample, which takes significant time (several minutes per sample) and API costs. For this demo, we'll use cached articles from `outputs/evals-pro`.


Define constants:

In [26]:
DATASET_NAME = "brown-course-lessons"
CACHE_DIR = OUTPUTS_DIR / "evals-pro"  # Here, we store the generated articles from the entire dataset
# to avoid recomputing them for this example.
METRICS_TO_USE = ["follows_gt", "user_intent"]
WORKERS = 1

Articles cached at:

In [27]:
CACHE_DIR

PosixPath('outputs/evals-pro')

Now, let's define the evaluation task and evaluation metrics as we've learnt in this Notebook:

In [28]:
from brown.evals.tasks import create_evaluation_task
from brown.models.config import ModelConfig
from brown.observability.evaluation import evaluate

evaluation_task = create_evaluation_task(
    cache_dir=CACHE_DIR,
    read_from_cache=True,  # Use cached articles from the `CACHE_DIR`
    clean_cache=False,
    debug=False,
)

model = SupportedModels.GOOGLE_GEMINI_25_FLASH
model_config = ModelConfig(temperature=0.0, thinking_budget=int(1024 * 0.5), include_thoughts=False, max_retries=3)
evaluation_metrics = build_evaluation_metrics(METRICS_TO_USE, model, model_config)

print("Evaluation Configuration:")
print(f"  - Dataset: {DATASET_NAME}")
print(f"  - Cache directory: {CACHE_DIR}")
print(f"  - Metrics: {METRICS_TO_USE}")
print(f"  - Workers: {WORKERS}")


Evaluation Configuration:
  - Dataset: brown-course-lessons
  - Cache directory: outputs/evals-pro
  - Metrics: ['follows_gt', 'user_intent']
  - Workers: 1


Define splits:

In [29]:
val_dataset_item_names = ["Lesson 10: Memory"]
test_dataset_item_names = [
    "Lesson 2: Workflows vs. Agents",
    "Lesson 3: Context Engineering",
    "Lesson 5: Workflow Patterns",
    "Lesson 6: Tools",
    "Lesson 8: ReAct Practice",
    "Lesson 9: Retrieval-Augmented Generation (RAG)",
    "Lesson 11: Multimodal Data",
]

SPLITS = {
    "val": val_dataset_item_names,
    "test": test_dataset_item_names,
}

## 12. Computing the Alignment Score on the Validation Split

Now that we have all the pieces in place, let's run the AI evals pipeline on the validation split to compute the alignment score, which checks whether the LLM judge aligns with a domain expert's human judgment.

In [30]:
from loguru import logger

split = "val"

logger.info(f"Starting evaluation with dataset: {DATASET_NAME} on the `{split}` split")
evaluate(
    dataset_name=DATASET_NAME,
    metrics=evaluation_metrics,
    evaluation_task=evaluation_task,
    llm_judge_config={"model": str(model), **model_config.model_dump()},
    workers=WORKERS,
    dataset_item_names=SPLITS[split],
    split=split,
)
logger.success("Evaluation completed successfully!")

2026-01-03 15:36:51.176 | INFO     | __main__:<module>:5 - Starting evaluation with dataset: brown-course-lessons on the `val` split
2026-01-03 15:36:53.391 | INFO     | brown.observability.evaluation:evaluate:56 - Successfuly filtered dataset based on the provided dataset item names.
2026-01-03 15:36:53.392 | INFO     | brown.observability.evaluation:evaluate:57 - Evaluating 1/8 dataset items.
2026-01-03 15:36:53.392 | INFO     | brown.observability.evaluation:evaluate:61 - Starting evaluation...
2026-01-03 15:36:53.394 | INFO     | brown.observability.evaluation:evaluate:70 - Evaluation details:
2026-01-03 15:36:53.395 | INFO     | brown.observability.evaluation:evaluate:71 - Dataset: brown-course-lessons
2026-01-03 15:36:53.395 | INFO     | brown.observability.evaluation:evaluate:72 - Metrics: ['FollowsGTMetricLLMJudge', 'UserIntentMetricLLMJudge']


Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

2026-01-03 15:36:54.202 | INFO     | brown.evals.tasks:evaluation_task:60 - Processing evaluation sample: Lesson 10: Memory
2026-01-03 15:36:54.204 | SUCCESS  | brown.evals.tasks:__run:136 - Successfully read article from cache at `outputs/evals-pro/data/10_memory_knowledge_access/article.md`
OPIK: Started logging traces to the "Default Project" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=019b8413-45bf-7fb9-9da6-e764ab01a9cb&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


╭─ brown-course-lessons (1 samples) ────────────╮
│                                               │
│ Total time:        00:00:26                   │
│ Number of samples: 1                          │
│                                               │
│ follows_gt_content: 0.8750 (avg)              │
│ follows_gt_flow: 0.5000 (avg)                 │
│ follows_gt_structure: 0.2500 (avg)            │
│ user_intent_guideline_adherence: 0.8571 (avg) │
│ user_intent_research_anchoring: 1.0000 (avg)  │
│                                               │
╰───────────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=570452;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=019b8413-429b-7d74-a6b8-8b7acf9576b7&dataset_id=019b4b30-8c7d-7428-bb0a-6da2a0d8a30a&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

2026-01-03 15:37:24.268 | SUCCESS  | __main__:<module>:15 - Evaluation completed successfully!


Now open the experiment in Opik using the dynamic link generated in the console output above, and inspect the results in Opik's dashboard.

For convenience, here are the results we got when we ran the experiment on our end:
```markdown
╭─ brown-course-lessons (1 samples) ────────────╮
│                                               │
│ Total time:        00:00:34                   │
│ Number of samples: 1                          │
│                                               │
│ follows_gt_content: 1.0000 (avg)              │
│ follows_gt_flow: 0.5714 (avg)                 │
│ follows_gt_structure: 0.5238 (avg)            │
│ user_intent_guideline_adherence: 0.8571 (avg) │
│ user_intent_research_anchoring: 1.0000 (avg)  │
│                                               │
╰───────────────────────────────────────────────╯
```

And here is how the experiment looks in Opik:

<img src="https://raw.githubusercontent.com/iusztinpaul/agentic-ai-engineering-course-data/main/images/30_ai_evals_offline_metrics_opik_validation_score.png" alt="30_ai_evals_offline_metrics_opik_validation_score"/>

And here is how the reasoning message looks in Opik, where you can visualize the results at the section level (you can hover over each sample and dimension to see it):

<img src="https://raw.githubusercontent.com/iusztinpaul/agentic-ai-engineering-course-data/main/images/30_ai_evals_offline_metrics_opik_validation_reasoning_message.png" alt="30_ai_evals_offline_metrics_opik_validation_reasoning_message"/>

At this step, using the validation split, we want to align the LLM Judges with our human judgment. For simplicity, let's assume that we want to do that only for the following GT metric.

To do that, as the validation split contains only `Lesson 10`, you will open the [cached generated article](https://github.com/iusztinpaul/agentic-ai-engineering-course-data/blob/main/data/outputs/evals-pro/data/10_memory_knowledge_access/article.md) and the [ground article](https://github.com/iusztinpaul/agentic-ai-engineering-course-data/blob/main/data/inputs/evals/dataset/data/10_memory_knowledge_access/article_ground_truth.md) side by side.

Then you will label this sample, writing down the scores and the reasons behind the scores for the content, flow, and structure metrics scoped under the GT metric.

Next, you will open the experiment in Opik's dashboard using the dynamically generated link from the function above and add the scores and reason messages computed by the LLM judge next to the ones you manually labeled.

Finally, you will compare your results with those from the LLM Judge. Since we use only binary metrics at the section level, we compute the alignment score using the following formula, which ranges from `[0, 1]`: Hence, we can consider it as an alignment percentage:

$$ \text{Agreement \%} = \frac{\sum_{j=1}^{n} [manualScore_j = LLMJudgeScore_j]}{n} \times 100 $$

The formula computes the average number of matching scores between your manual labeling and the LLM judge. If the alignment is close to 100%, then congratulations, your LLM Judge is aligned with human judgment. Otherwise, you still have to improve the LLM judge until the scores align.

Only when the alignment percentage approaches 100% can you start relying on your LLM Judge.

### Computing the Alignment Scores

To show you how aligning works in pratice, let's compute the alignment scores on `Lesson 10 - Memory` on the follows ground truth metric.

To do that, we:
- opened the [cached generated memory article](https://github.com/iusztinpaul/agentic-ai-engineering-course-data/blob/main/data/outputs/evals-pro/data/10_memory_knowledge_access/article.md) (we have to fix the generated article as well to have all the components fixed except the LLM Judge itself)
- opened the [ground truth memory article](https://github.com/iusztinpaul/agentic-ai-engineering-course-data/blob/main/data/inputs/evals/dataset/data/10_memory_knowledge_access/article_ground_truth.md) from the AI evals dataset
- labeled the difference between the two (both scores and reason messages)
- ran an experiment using the validation split
- added the LLM judge results along with the manual ones
- computed the alignemnt score using the formula above

Here are the results:

#### Content Metric Alignment

| Section                                                   |   Score Content | Reason Content                                                                                                                                                                     |   LLM Judge Score Content | LLM Judge Reason Content                                                                                                                                                                                                                                                                                                                                                                 |
|:----------------------------------------------------------|----------------:|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|--------------------------:|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Introduction                                              |               1 | Even if the examples are a bit different it both sections cover the same introductory ideas about memory for AI agents.                                                            |                         1 | Both sections introduce the concept of memory in AI agents, highlighting the stateless nature of LLMs and the role of the context window. They both set the stage for the lesson by explaining why memory is a necessary engineering solution.                                                                                                                                           |
| The Layers of Memory: Internal, Short-Term, and Long-Term |               0 | Even if it covers the same ideas about internal, short, and long-term memory, the generated section misses the ideas on combining them into a unified system and reference to RAG. |                         1 | Both sections accurately describe the three layers of memory: Internal Knowledge, Short-Term Memory (Context Window), and Long-Term Memory, explaining their distinct functions and interplay.                                                                                                                                                                                           |
| Long-Term Memory: Semantic, Episodic, and Procedural      |               1 | Both sections cover the same ideas around semantic, episodic and procedural memory.                                                                                                |                         1 | Both sections correctly categorize long-term memory into Semantic, Episodic, and Procedural types, providing clear definitions and examples for each.                                                                                                                                                                                                                                    |
| Storing Memories: Pros and Cons of Different Approaches   |               1 | Both sections cover the same ideas around storing the memory as raw strings, structured entities or knowledge graphs.                                                              |                         1 | Both sections discuss the same three approaches for storing memories: Raw Strings (Vector Databases), Structured Entities (JSON), and Knowledge Graphs, outlining the pros and cons for each.                                                                                                                                                                                            |
| Memory Implementations With Code Examples                 |               1 | Both sections cover examples on implementing semantic, episodic and procedural memory with mem0.                                                                                   |                         1 | Both sections provide code examples for implementing Semantic, Episodic, and Procedural memory. However, the generated section uses the `mem0` library, while the expected output also uses `mem0` but presents the code and explanations in a slightly different structure and with more detailed comments. The core concepts demonstrated are the same.                                |
| Real-World Challenges                                     |               0 | The generated section adds an extra topic on "The Human Factor" that's entirely missing within the expected section.                                                               |                         0 | The generated section covers similar themes, such as re-evaluating compression due to larger context windows and designing for the product. However, it presents these as two distinct points, whereas the expected output integrates them into a more cohesive narrative. The generated section also adds a "The Human Factor" subsection, which is not present in the expected output. |
| Conclusion                                                |               1 | Both sections cover the same ideas presenting what we've learnt within the article.                                                                                                |                         1 | Both sections provide a summary of the lesson's key takeaways, emphasizing that memory transforms agents from stateless tools to adaptive partners and that current memory systems are an engineering workaround for the lack of continual learning in LLMs.                                                                                                                             |
| References                                                |               0 | The generated section misses 10 links.                                                                                                                                             |                         1 | The generated section provides a list of references, similar in purpose to the expected output. However, the specific references cited are different.                                                                                                                                                                                                                                    |

**LLM Judge Alignment - Content Metric: 75.00%**

#### Flow Metric Alignment

| Section                                                   |   Score Flow | Reason Flow                                                                                                                                                                                                                                                                                                                                                       |   LLM Judge Score Flow | LLM Judge Reason Flow                                                                                                                                                                                                                                                                                                                                                                                            |
|:----------------------------------------------------------|-------------:|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-----------------------:|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Introduction                                              |            0 | Covers the core ideas in the same order, such as LLMs forgetting past interactions, using the context window to anchor the conversation, and the limitations of the context window. Still, the last sentence acting as the transition differs: the generated section makes a quick outline, while the expected section highlights its characteristics in biology. |                      0 | The generated section follows a similar flow, starting with a personal anecdote, explaining the statelessness of LLMs, and introducing memory as the solution. However, it omits the specific mention of OpenAI's ChatGPT memory feature, which is present in the expected output.                                                                                                                               |
| The Layers of Memory: Internal, Short-Term, and Long-Term |            0 | The generated section starts with a comparison to biology, while the expected section jumps straight to the three memory types, with a small transition.                                                                                                                                                                                                          |                      1 | The flow of ideas is consistent in both sections, starting with an introduction to the layers, defining each one, and concluding with a diagram that illustrates their relationship.                                                                                                                                                                                                                             |
| Long-Term Memory: Semantic, Episodic, and Procedural      |            0 | The generated section misses the last transition sentence.                                                                                                                                                                                                                                                                                                        |                      1 | The flow is identical, with both sections introducing the three categories and then dedicating a subsection (using H3 headers) to explain each one in the same order: Semantic, Episodic, and Procedural.                                                                                                                                                                                                        |
| Storing Memories: Pros and Cons of Different Approaches   |            0 | Even if both sections have an introductory paragraph, the generated section text misses most of the ideas such as the discussion on tools for agents to interact with the memory.                                                                                                                                                                                 |                      0 | The generated section follows a similar logical flow, introducing the topic and then detailing each storage approach. However, it omits the introductory paragraph from the expected output that discusses the challenge of updating memories and the role of LLMs in managing `ADD/UPDATE/DELETE` operations.                                                                                                   |
| Memory Implementations With Code Examples                 |            0 | The generated section adds an extra "extraction prompt" example within the semantic memory subsection that is not present within the expected section.                                                                                                                                                                                                            |                      0 | The generated section's flow is logical, covering setup and then each memory type. However, it differs from the expected output, which integrates the helper function definitions within the setup and provides more detailed, step-by-step explanations for each code block's output. The generated section also misses the introductory paragraph that frames the code examples and links to a Colab notebook. |
| Real-World Challenges                                     |            0 | The generated section adds an extra "The Human Factor" that's missing within the expected section.                                                                                                                                                                                                                                                                |                      0 | The flow is different. The generated section breaks the discussion into "Re-evaluating Compression," "Designing for the Product," and "The Human Factor." The expected output has a more integrated flow, discussing compression and product design as interconnected challenges without the distinct separation.                                                                                                |
| Conclusion                                                |            1 | Both sections follow the same flow, presenting what we've learnt within the article, wrapping up with a transition to the next lesson.                                                                                                                                                                                                                            |                      1 | Both sections follow a similar flow, summarizing the main points of the lesson and providing a forward-looking statement about the next topics in the course (RAG and Multimodal data).                                                                                                                                                                                                                          |
| References                                                |            1 | -                                                                                                                                                                                                                                                                                                                                                                 |                      1 | Both sections present the references as a numbered list, which is a consistent flow.                                                                                                                                                                                                                                                                                                                             |

**LLM Judge Alignment - Flow Metric: 75.00%**


#### Structure Metric Alignment

| Section                                                   |   Score Structure | Reason Structure                                                                                                                         |   LLM Judge Score Structure | LLM Judge Reason Structure                                                                                                                                                                                                                                                                                                         |
|:----------------------------------------------------------|------------------:|:-----------------------------------------------------------------------------------------------------------------------------------------|----------------------------:|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Introduction                                              |                 1 | Both sections are structured the same as simple text paragraphs.                                                                         |                           1 | Both sections use similar paragraph structures and bolding for emphasis. The generated section correctly uses citation formatting.                                                                                                                                                                                                 |
| The Layers of Memory: Internal, Short-Term, and Long-Term |                 1 | Both sections enumerate the three memory types within simple paragraphs with bolded key words.                                           |                           1 | Both sections use bolding for key terms and include a Mermaid diagram to visualize the concept. The formatting of the diagram and its caption is consistent.                                                                                                                                                                       |
| Long-Term Memory: Semantic, Episodic, and Procedural      |                 1 | Both sections structured the three long-term memory types under H3 headers.                                                              |                           1 | Both sections use H3 headers for the sub-categories, maintaining a consistent structure. The use of bolding and italics for emphasis is also similar.                                                                                                                                                                              |
| Storing Memories: Pros and Cons of Different Approaches   |                 0 | The generated section uses numbered H3 headers instead of simple H3 headers as in the expected section.                                  |                           0 | The generated section uses numbered H3 headers for each approach, which differs from the expected output's use of unnumbered H3 headers. Additionally, the expected output includes a detailed comparison table, which is missing from the generated section                                                                       |
| Memory Implementations With Code Examples                 |                 1 | Both sections use numebered lists to present the code and divide the impelemntation examples using H3 headers.                           |                           0 | The generated section uses H3 headers for each implementation, which is a different structure from the expected output's H3 headers for each memory type. The expected output also includes a "Tip" callout box, which is absent in the generated version. The formatting of code blocks and outputs is similar but not identical. |
| Real-World Challenges                                     |                 1 | Both sections divide the ides under H3 headers and simple text paragraphs.                                                               |                           0 | The structure is different. The generated section uses H3 headers for its points, while the expected output uses a single H2 header for the entire section and presents the content as continuous prose.                                                                                                                           |
| Conclusion                                                |                 0 | Both sections are structured as simple text paragraphs, but the generated sections bold the word "Retrieval-Augmented-Generation (RAG)". |                           1 | The paragraph structure and overall formatting are consistent between both sections.                                                                                                                                                                                                                                               |
| References                                                |                 0 | The references from the generated section do not follow the same APA format.                                                             |                           0 | The formatting of the references is different. The generated output uses a format like "1. Title. (Date). Source. URL", while the expected output uses "[<number>] [Title](URL)". The generated section also includes an "Images" subsection at the end, which is not in the expected output.                                      |

**LLM Judge Alignment - Structure Metric: 62.50%**


#### Next Steps

As you can see, the scores are far from 100%. So what next?

We have to look where the scores do not match, adapt the LLM Judges by either improving the few-shot examples or the system prompt and retry until the scores get closer to 100%.

Still, if you look only at the difference in scores, it's tough to figure out what to improve. That's why we labeled the reason message as well. Looking at it, we have a lot of signals on knowing what to improve.

For example, if we look at `The Layers of Memory: Internal, Short-Term, and Long-Term` within the `flow metric` table, we can see that the LLM judge gave a score of 1 because it ignored the transition paragraph, which is wrong. Thus, we know that we should add this rule to the LLM judge system prompt.

## 13. Computing the Baseline Score on the Test Split

For the sake of this notebook, let's assume that this alignment score is acceptable.

Thus, let's run the evaluation on the test split to compute the baseline score:

In [31]:
split = "test"

logger.info(f"Starting evaluation with dataset: {DATASET_NAME} on the `{split}` split")
evaluate(
    dataset_name=DATASET_NAME,
    metrics=evaluation_metrics,
    evaluation_task=evaluation_task,
    llm_judge_config={"model": str(model), **model_config.model_dump()},
    workers=WORKERS,
    dataset_item_names=SPLITS[split],
    split=split,
)
logger.success("Evaluation completed successfully!")

2026-01-03 15:37:24.302 | INFO     | __main__:<module>:3 - Starting evaluation with dataset: brown-course-lessons on the `test` split
2026-01-03 15:37:36.332 | INFO     | brown.observability.evaluation:evaluate:56 - Successfuly filtered dataset based on the provided dataset item names.
2026-01-03 15:37:36.334 | INFO     | brown.observability.evaluation:evaluate:57 - Evaluating 7/8 dataset items.
2026-01-03 15:37:36.334 | INFO     | brown.observability.evaluation:evaluate:61 - Starting evaluation...
2026-01-03 15:37:36.334 | INFO     | brown.observability.evaluation:evaluate:70 - Evaluation details:
2026-01-03 15:37:36.335 | INFO     | brown.observability.evaluation:evaluate:71 - Dataset: brown-course-lessons
2026-01-03 15:37:36.338 | INFO     | brown.observability.evaluation:evaluate:72 - Metrics: ['FollowsGTMetricLLMJudge', 'UserIntentMetricLLMJudge']


Evaluation:   0%|          | 0/7 [00:00<?, ?it/s]

2026-01-03 15:37:39.885 | INFO     | brown.evals.tasks:evaluation_task:60 - Processing evaluation sample: Lesson 11: Multimodal Data
2026-01-03 15:37:39.887 | SUCCESS  | brown.evals.tasks:__run:136 - Successfully read article from cache at `outputs/evals-pro/data/11_multimodal/article.md`
2026-01-03 15:38:09.216 | INFO     | brown.evals.tasks:evaluation_task:60 - Processing evaluation sample: Lesson 9: Retrieval-Augmented Generation (RAG)
2026-01-03 15:38:09.218 | SUCCESS  | brown.evals.tasks:__run:136 - Successfully read article from cache at `outputs/evals-pro/data/09_RAG/article.md`
2026-01-03 15:38:30.887 | INFO     | brown.evals.tasks:evaluation_task:60 - Processing evaluation sample: Lesson 8: ReAct Practice
2026-01-03 15:38:30.889 | SUCCESS  | brown.evals.tasks:__run:136 - Successfully read article from cache at `outputs/evals-pro/data/08_react_practice/article.md`
2026-01-03 15:38:54.085 | INFO     | brown.evals.tasks:evaluation_task:60 - Processing evaluation sample: Lesson 6:

╭─ brown-course-lessons (7 samples) ────────────╮
│                                               │
│ Total time:        00:03:03                   │
│ Number of samples: 7                          │
│                                               │
│ follows_gt_content: 0.9597 (avg)              │
│ follows_gt_flow: 0.3091 (avg)                 │
│ follows_gt_structure: 0.1882 (avg)            │
│ user_intent_guideline_adherence: 0.4796 (avg) │
│ user_intent_research_anchoring: 0.8075 (avg)  │
│                                               │
╰───────────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=677661;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=019b8413-ea53-7e51-9c35-b45ed96134c3&dataset_id=019b4b30-8c7d-7428-bb0a-6da2a0d8a30a&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

2026-01-03 15:40:42.877 | SUCCESS  | __main__:<module>:13 - Evaluation completed successfully!


Now open the experiment in Opik using the dynamic link generated in the console output above, and inspect the results in Opik's dashboard.

For convenience, here are the results we got when we ran the experiment on our end:
```markdown
╭─ brown-course-lessons (7 samples) ────────────╮
│                                               │
│ Total time:        00:03:03                   │
│ Number of samples: 7                          │
│                                               │
│ follows_gt_content: 0.8962 (avg)              │
│ follows_gt_flow: 0.4127 (avg)                 │
│ follows_gt_structure: 0.1086 (avg)            │
│ user_intent_guideline_adherence: 0.7468 (avg) │
│ user_intent_research_anchoring: 0.8075 (avg)  │
│                                               │
╰───────────────────────────────────────────────╯
```

And here is how the experiment looks in Opik:

<img src="https://raw.githubusercontent.com/iusztinpaul/agentic-ai-engineering-course-data/main/images/30_ai_evals_offline_metrics_opik_baseline_score.png" alt="30_ai_evals_offline_metrics_opik_baseline_score" height=600/>

And here is how the reasoning message looks in Opik, where you can visualize the results at the section level (you can hover over each sample and dimension to see it):

<img src="https://raw.githubusercontent.com/iusztinpaul/agentic-ai-engineering-course-data/main/images/30_ai_evals_offline_metrics_opik_baseline_score_reasoning_message.png" alt="30_ai_evals_offline_metrics_opik_baseline_score_reasoning_message" height=600/>

These scores are the baseline to compare against when we make future changes to our writing workflow.

## 14. Checking the Stability of the LLM Judge

Another criterion you should check before relying on your LLM judge is its stability. Why? Because we use LLMs to compute the metrics, we might get different scores after each run. Even if we did our best to stabilize these scores, for example, by using binary scores to make it easier for the LLM to choose one option, the LLM judge can still be unstable.

So, how can we compute the stability of the LLM judge?

As before, we fix all components on the [outputs/evals-pro](https://github.com/iusztinpaul/agentic-ai-engineering-course-data/tree/main/data/outputs/evals-pro/data) cached generated articles and evaluation dataset, and run the experiment multiple times to see how much the scores vary. The idea is that, by keeping everything the same within the AI evals pipeline, the results differ purely due to the stochastic nature of the LLMs.

In our use case, we did that 5 times and got the following results:

<img src="https://raw.githubusercontent.com/iusztinpaul/agentic-ai-engineering-course-data/main/images/30_ai_evals_offline_metrics_practice_llm_judge_variation.png" alt="LLM Judge Variation" height=600/>

As you can see in the image above, the lines for each metric are pretty flat. It's not 100% flat, but it doesn't vary much, which means our LLM judges are stable, though not 100% stable.

To quantify this variation, we computed the standard deviation for each metric:

| Metric | Standard Deviation |
| :--- | :--- |
| follows_gt_content | 0.0316042 |
| follows_gt_flow | 0.0432000 |
| follows_gt_structure | 0.0180826 |
| user_intent_guideline_adherence | 0.0380109 |
| user_intent_research_anchoring | 0.0268774 |


#### What Next?

As you can see, the standard deviation is not 0, which means the LLM judges are not purely stable between runs. So what can you do?

There are 2 options:
1. You can use this standard deviation to understand when a difference in a particular metric is statistically significant. For example, for `follows_gt_content`, we have a standard deviation of ~0.031. Thus, if we have a difference of <=0.031 between the baseline and the change you made to the AI app, it's probably probabilistically insignificant.
2. You can improve the LLM judge and stabilize it by trying to get this number close to 0. Which can be more time-consuming, but it makes your system more trustworthy. You can do that by making it easier for the LLM to decide between 0 and 1. To do this, you need to review the data, label more samples, and identify which scenarios are fuzzy and need clarification.

Probably the best strategy is to start with option 1 to get started and strive for option 2 in the long run as you keep using the system.

Now... How can we use these AI Evals in practice? How can we integrate them into our real-world processes? We do that through the optimization loop.

### The Optimization Loop

What we just did is called running an **experiment**. For a particular state of our AI application and a given evaluation dataset, we computed scores reflecting how well our system performs.

Thus, whenever you plan to do changes to your AI application you have to follow the next strategy, known as the **optimization loop:**

1. Run an experiment to get baseline metrics
2. Make changes to your AI app (prompts, models, parameters)
3. Run another experiment
4. Compare metrics to see if the change improved or degraded performance
5. Repeat until satisfied

Now that we have clarity on how our system works (or doesn't work), we can start tweaking our AI app while using the metrics as our north star.

## 15. Exploring the Experiments Inside Opik

Here is a short video presenting in more detail how you can leverage Opik's dashboard to zoom in on each experiment or compare multiple experiments:

<a href="https://youtu.be/6XDzz9v0pao" target="_blank">
   <img src="https://img.youtube.com/vi/6XDzz9v0pao/maxresdefault.jpg" alt="LLM Judge Variation" height="400"/>
</a>

## 16. Conclusion

AI evals are critical for building reliable AI applications. Without metrics, you're flying blind, making changes based on gut feeling and vibe checking rather than evidence.

### What We Learned

In this lesson, we covered:

- **LLM Judge Architecture**: How to build LLM judges that use structured output to evaluate articles at the section level with binary metrics
- **FollowsGT Metric**: Compares generated articles against ground truth across content, flow, and structure dimensions
- **UserIntent Metric**: Checks if articles follow the guideline and are anchored in the research (hallucination detection)
- **Few-Shot Examples**: How to manually create labeled examples to "train" the LLM judges
- **Evaluation Pipeline**: How to run experiments using Opik and the Brown agent

### Wrapping Up AI Evals

With this lesson, we've wrapped up the AI evals section of the course. Now we move on to other ops aspects such as deploying the AI apps and building CI/CD pipelines.

### Practicing Ideas

1. **Test LLM Judge Stability**: We showed the LLM Judge stability on the article generated with `gemini-pro` using the cached articles from [outputs/evals-pro](https://github.com/iusztinpaul/agentic-ai-engineering-course-data/tree/main/data/outputs/evals-pro/data). Now repeat the same process using the cached articles from the [outputs/evals-flash](https://github.com/iusztinpaul/agentic-ai-engineering-course-data/tree/main/data/outputs/evals-flash/data) directory. Run the LLM judges 5 times with all the inputs fixed. Measure the stability of the scores across runs. Are the results more stable or worse when using `flash`?

2. **Compute the Alignment Score on the User Intent Metrics**: Use the Memory lesson from the validation split and manually score each section on the user intent metrics, such as article guideline adherence and research anchoring. Compare your scores with the LLM judge scores to see how aligned they are.

3. **Iterate on Brown**: Make changes to the Brown AI app (prompts, model, temperature) and rerun the AI evals. See if your change improves or degrades the metrics?

### Useful Resources

- **Brown Package**: Explore `lessons/writing_workflow`
- **Evaluation Script**: Check `lessons/writing_workflow/scripts/brown_run_eval.py`
- **Configuration Examples**: Check `configs/` for different configurations
- **Test Data**: Use `inputs/tests/` for additional testing scenarios

### 💡 Run Brown as a Standalone Python Project

Remember that you can also run `brown` as a standalone Python project by going to `lessons/writing_workflow/` and following the instructions from there. We support running the evaluation within the `scripts/brown_run_eval.py` Python script.
